In [1]:
using ITensors
ITensors.enable_debug_checks()

In [2]:
import Base: *,+,-,/,keys,iterate,length,getindex,show,repr,zero
mutable struct ket <:Number
    storage::Dict
end
#
# Singleton constructor
ket(state)=ket(Dict(string(state)=>1))
#
# Empty state
zero(::ket)=ket(Dict())
ket()=ket(Dict())
#
# Iteration and indexing
iterate(k::ket)=iterate(k.storage)
iterate(k::ket,n)=iterate(k.storage,n)
length(k::ket)=length(k.storage)
keys(k::ket) = keys(k.storage)
getindex(k::ket,val)=getindex(k.storage,val)
#
# Multiplication as outer product
*(k1::ket,k2::ket) = ket(Dict([v1*v2=>s1*s2 for (v1,s1) in k1 for (v2,s2) in k2 ]))
# Multiplication with scalar
*(c::Number,k::ket)=ket(Dict([v=>c*s for (v,s) in k]))
*(k::ket,c::Number)=c*k
/(k::ket,c::Number)=(1/c)*k
#
# Sums
function +(k1::ket,k2::ket) 
    d=copy(k1.storage)
    for key in keys(k2) 
        if (key in keys(k1))
            d[key]+=k2[key]
        else
            d[key]=k2[key]
        end
        if d[key]===zero(d[key])
            delete!(d,key)
        end
    end
    return ket(d)
end
function -(k1::ket,k2::ket) 
    d=copy(k1.storage)
    for key in keys(k2) 
        if (key in keys(k1))
            d[key]-=k2[key]
        else
            d[key]=-k2[key]
        end
        if d[key]===zero(d[key])
            delete!(d,key)
        end
    end
    return ket(d)
end
function show(io::IO,::MIME"text/latex",k::ket)
    len=length(k)
    j=1
    outputstring="\$\$"
    for (label,val) in k
        if val===1
            if j>1
                outputstring *="+"
            end
        else
            try
                sn=sign(val)
                if sn>0
                    if j>1
                        outputstring *="+"
                    end    
                    outputstring*=string(val)
                elseif sn<0
                    outputstring*=string(val)
                else
                    if j>1
                        outputstring *="+"
                    end
                    outputstring*="("*string(val)*")"
                end
            catch er #probably a comparison with zero problem
                if j>1
                        outputstring *="+"
                end
                outputstring*="("*string(val)*")"
            end
        end
        outputstring*="|"*string(label)*"\\rangle"
        j+=1
    end
    outputstring*="\$\$"
    print(io,outputstring)
end
function repr(k::ket)
    len=length(k)
    j=1
    outputstring=""
    for (label,val) in k
        if val===1
            if j>1
                outputstring *="+"
            end
        else
            try
                sn=sign(val)
                if sn>0
                    if j>1
                        outputstring *="+"
                    end    
                    outputstring*=string(val)
                elseif sn<0
                    outputstring*=string(val)
                else
                    if j>1
                        outputstring *="+"
                    end
                    outputstring*="("*string(val)*")"
                end
            catch er #probably a comparison with zero problem
                if j>1
                        outputstring *="+"
                end
                outputstring*="("*string(val)*")"
            end
        end
        outputstring*="|"*string(label)*"⟩"
        j+=1
    end
    return outputstring
end
function show(io::IO,k::ket)
    outputstring=repr(k)
    print(io,outputstring)
end
# self-type conversion
ket(k::ket)=k
# alternate display
function tabledisplay(k::ket)
    for pair in k
        println(pair[1]*" "*string(pair[2]))
    end
end
up=ket("↑")
dn=ket("↓")
empty=ket("⭕️") #represents an empty site
full=ket("🔵") #represents a full site
kz=ket() # the 'zero' ket
#different form of visual representations
function scalarpp(T::ITensor,localstates::Array{ket})
    ar=array(T) #unpack raw tensor -- as it is easier to work with
    result=ket() #empty ket to use as result
    for ci in CartesianIndices(ar) #loop over indices of the array
        nonzero=true
        try
            if abs(ar[ci])<1E-6 # only do something if the array's entry is above a threshold
                nonzero=false
            end
        catch
            nonzero=true
        end
        if nonzero
            newterm=ket("") # create trivial ket
            for site in ci.I
                newterm=newterm*localstates[site]  # add site information to ket
            end
            result+= ar[ci]*newterm # add ket to result
        end
    end
    result
end
vectorpp(T::ITensor,bondindex::Index,localstates::Array{ket})=
    [scalarpp(T*dag(onehot(bondval)),localstates) for bondval in eachindval(bondindex)]
matrixpp(T::ITensor,leftbondindex::Index,rightbondindex::Index,localstates::Array{ket})=
   [scalarpp(T*dag(onehot(leftbondval,rightbondval)),localstates) for 
        leftbondval in eachindval(leftbondindex),
        rightbondval in eachindval(rightbondindex)]

matrixpp (generic function with 1 method)

In [3]:
up

|↑⟩

In [4]:
site=Index(QN("n",0)=>1,QN("n",1)=>1,tags="site")

(dim=2|id=75|"site") <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1

In [5]:
dsite=dag(site)

(dim=2|id=75|"site") <In>
 1: QN("n",0) => 1
 2: QN("n",1) => 1

In [6]:
site1=addtags(site,"s=1")
site2=addtags(site,"s=2")
psi2=ITensor(site1,site2)
psi2[site1=>1,site2=>2]=0.2+0.5im
psi2[site1=>2,site2=>1]=0.3+0.4im
@show psi2;

psi2 = ITensor ord=2
Dim 1: (dim=2|id=75|"s=1,site") <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
Dim 2: (dim=2|id=75|"s=2,site") <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
NDTensors.BlockSparse{ComplexF64, Vector{ComplexF64}, 2}
 2×2
Block(1, 2)
 [1:1, 2:2]
 0.2 + 0.5im

Block(2, 1)
 [2:2, 1:1]
 0.3 + 0.4im


In [7]:
@show dag(psi2)

dag(psi2) = ITensor ord=2
Dim 1: (dim=2|id=75|"s=1,site") <In>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
Dim 2: (dim=2|id=75|"s=2,site") <In>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
NDTensors.BlockSparse{ComplexF64, Vector{ComplexF64}, 2}
 2×2
Block(1, 2)
 [1:1, 2:2]
 0.2 - 0.5im

Block(2, 1)
 [2:2, 1:1]
 0.3 - 0.4im


ITensor ord=2
(dim=2|id=75|"s=1,site") <In>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
(dim=2|id=75|"s=2,site") <In>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
NDTensors.BlockSparse{ComplexF64, Vector{ComplexF64}, 2}

In [8]:
only(psi2*dag(psi2))

0.54 + 0.0im

In [9]:
kron([empty full],[empty full])*tensors(removeqns(psi2))

1-element Vector{ket}:
 (0.2 + 0.5im)|🔵⭕️⟩+(0.3 + 0.4im)|⭕️🔵⟩

In [10]:
#making a new state due to the improper quantum flux
psi2b=ITensor(site1,site2)
psi2b[site1=>2,site2=>2]=1
@show psi2b

psi2b = ITensor ord=2
Dim 1: (dim=2|id=75|"s=1,site") <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
Dim 2: (dim=2|id=75|"s=2,site") <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
NDTensors.BlockSparse{Int64, Vector{Int64}, 2}
 2×2
Block(2, 2)
 [2:2, 2:2]
 1


ITensor ord=2
(dim=2|id=75|"s=1,site") <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
(dim=2|id=75|"s=2,site") <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
NDTensors.BlockSparse{Int64, Vector{Int64}, 2}

In [11]:
num=ITensor([0 0;0 1],dsite,site')
array(num)

2×2 Matrix{Float64}:
 0.0  0.0
 0.0  1.0

In [12]:
@show num;

num = ITensor ord=2
Dim 1: (dim=2|id=75|"site") <In>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
Dim 2: (dim=2|id=75|"site")' <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
NDTensors.BlockSparse{Float64, Vector{Float64}, 2}
 2×2
Block(2, 2)
 [2:2, 2:2]
 1.0


In [13]:
num2=op([0 0;0 1],site)
num2==num

true

**Creating the annhilation operator**

In [14]:
a=op([0 1;0 0],site)
array(a)

2×2 Matrix{Float64}:
 0.0  1.0
 0.0  0.0

In [15]:
@show a;

a = ITensor ord=2
Dim 1: (dim=2|id=75|"site")' <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
Dim 2: (dim=2|id=75|"site") <In>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
NDTensors.BlockSparse{Float64, Vector{Float64}, 2}
 2×2
Block(1, 2)
 [1:1, 2:2]
 1.0


**calculating the commutator for a and n**

In [16]:
commutator=replaceprime(a*prime(num)-num*prime(a),2=>1)

ITensor ord=2
(dim=2|id=75|"site") <In>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
(dim=2|id=75|"site")' <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
NDTensors.BlockSparse{Float64, Vector{Float64}, 2}

In [17]:
array(commutator)

2×2 Matrix{Float64}:
  0.0  0.0
 -1.0  0.0

In [18]:
a=op([0 1;0 0],site)
a_herm=ITensor([0 0;1 0],dsite,site')
@show a_herm,a

(a_herm, a) = (ITensor ord=2
Dim 1: (dim=2|id=75|"site") <In>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
Dim 2: (dim=2|id=75|"site")' <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
NDTensors.BlockSparse{Float64, Vector{Float64}, 2}
 2×2
Block(2, 1)
 [2:2, 1:1]
 1.0, ITensor ord=2
Dim 1: (dim=2|id=75|"site")' <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
Dim 2: (dim=2|id=75|"site") <In>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
NDTensors.BlockSparse{Float64, Vector{Float64}, 2}
 2×2
Block(1, 2)
 [1:1, 2:2]
 1.0)


(ITensor ord=2
Dim 1: (dim=2|id=75|"site") <In>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
Dim 2: (dim=2|id=75|"site")' <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
NDTensors.BlockSparse{Float64, Vector{Float64}, 2}
 2×2
Block(2, 1)
 [2:2, 1:1]
 1.0, ITensor ord=2
Dim 1: (dim=2|id=75|"site")' <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
Dim 2: (dim=2|id=75|"site") <In>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
NDTensors.BlockSparse{Float64, Vector{Float64}, 2}
 2×2
Block(1, 2)
 [1:1, 2:2]
 1.0)

**Problem-1**

In [19]:
a=op([0 1;0 0],site)
a_herm=ITensor([0 0;1 0],dsite,site');
commutator=replaceprime(a*prime(a_herm)-a_herm*prime(a))
@show array(commutator)

array(commutator) = [0.0 0.0; 0.0 0.0]


2×2 Matrix{Float64}:
 0.0  0.0
 0.0  0.0

In [20]:
ITensor([3 0;0 4],site,dag(site'))

ITensor ord=2
(dim=2|id=75|"site") <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
(dim=2|id=75|"site")' <In>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
NDTensors.BlockSparse{Float64, Vector{Float64}, 2}

In [21]:
ITensor([0 1;1 0],site,dag(site'))

LoadError: Fluxes not all equal

**Naming operators**

In [22]:
i=Index(3,"spin1")

(dim=3|id=420|"spin1")

In [23]:
Szi1=op([-1 0 0;0 0 0;0 0 1],i)

ITensor ord=2 (dim=3|id=420|"spin1")' (dim=3|id=420|"spin1")
NDTensors.Dense{Float64, Vector{Float64}}

In [24]:
#naming the operator with somewhat obscure notation
ITensors.op(::OpName"Sz",::SiteType"spin1")=[-1 0 0;0 0 0;0 0 1]

In [25]:
Szi2=op("Sz",i)

ITensor ord=2 (dim=3|id=420|"spin1")' (dim=3|id=420|"spin1")
NDTensors.Dense{Float64, Vector{Float64}}

In [26]:
Szi1==Szi2

true

In [27]:
@show Szi2

Szi2 = ITensor ord=2
Dim 1: (dim=3|id=420|"spin1")'
Dim 2: (dim=3|id=420|"spin1")
NDTensors.Dense{Float64, Vector{Float64}}
 3×3
 -1.0  0.0  0.0
  0.0  0.0  0.0
  0.0  0.0  1.0


ITensor ord=2 (dim=3|id=420|"spin1")' (dim=3|id=420|"spin1")
NDTensors.Dense{Float64, Vector{Float64}}

In [28]:
site_1=Index(QN("n",0)=>1,QN("n",1)=>1,tags="site_1")

(dim=2|id=569|"site_1") <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1

In [29]:
ITensors.op(::OpName"Num",::SiteType"site_1")=[0 0;0 1]
@show op("Num",site_1);

op("Num", site_1) = ITensor ord=2
Dim 1: (dim=2|id=569|"site_1")' <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
Dim 2: (dim=2|id=569|"site_1") <In>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
NDTensors.BlockSparse{Float64, Vector{Float64}, 2}
 2×2
Block(2, 2)
 [2:2, 2:2]
 1.0


In [30]:
ITensors.op(::OpName"a",::SiteType"site_1")=[0 0;0 1]
@show op("a",site_1);

op("a", site_1) = ITensor ord=2
Dim 1: (dim=2|id=569|"site_1")' <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
Dim 2: (dim=2|id=569|"site_1") <In>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
NDTensors.BlockSparse{Float64, Vector{Float64}, 2}
 2×2
Block(2, 2)
 [2:2, 2:2]
 1.0


In [31]:
ITensors.op(::OpName"ad",::SiteType"site_1")=[0 0;0 1]
@show op("ad",site_1)

op("ad", site_1) = ITensor ord=2
Dim 1: (dim=2|id=569|"site_1")' <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
Dim 2: (dim=2|id=569|"site_1") <In>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
NDTensors.BlockSparse{Float64, Vector{Float64}, 2}
 2×2
Block(2, 2)
 [2:2, 2:2]
 1.0


ITensor ord=2
(dim=2|id=569|"site_1")' <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
(dim=2|id=569|"site_1") <In>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
NDTensors.BlockSparse{Float64, Vector{Float64}, 2}

In [32]:
site1=Index(QN("n",0)=>1,QN("n",1)=>1,tags="site_1")
# site1=addtags(site1,"s=1")
site2=addtags(site1,"s=2")
site3=addtags(site1,"s=3")

(dim=2|id=70|"s=3,site_1") <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1

In [33]:
bond1=Index(QN("n",0)=>1,QN("n",1)=>1,tags="bond1")
bond2=settags(bond1,"bond2")

(dim=2|id=198|"bond2") <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1

In [34]:
A1=onehot(site1=>1,bond1=>1)+onehot(site1=>2,bond1=>2)
vectorpp(A1,bond1,[empty,full])

2-element Vector{ket}:
 1.0|⭕️⟩
 1.0|🔵⟩

In [35]:
A2=ITensor([1 0;0 1],dag(bond1),bond2)*onehot(site2=>1)+ITensor([0 1;0 0],dag(bond1),bond2)*onehot(site2=>2)
matrixpp(A2,dag(bond1),bond2,[empty,full])

2×2 Matrix{ket}:
 1.0|⭕️⟩  1.0|🔵⟩
          1.0|⭕️⟩

In [36]:
A3=onehot(site3=>1,dag(bond2)=>2)+onehot(site3=>2,dag(bond2)=>1)
vectorpp(A3,dag(bond2),[empty,full])

2-element Vector{ket}:
 1.0|🔵⟩
 1.0|⭕️⟩

In [37]:
psi=MPS([site1,site2,site3])
psi[1]=A1
psi[2]=A2
psi[3]=A3
orthogonalize!(psi,1); 

LoadError: Fluxes not all equal

In [38]:
vectorpp(psi[1],linkind(psi,1),[empty,full])

2-element Vector{ket}:
 1.0|⭕️⟩
 1.0|🔵⟩

In [39]:
matrixpp(psi[2],dag(linkind(psi,1)),linkind(psi,2),[empty,full])

2×2 Matrix{ket}:
 1.0|⭕️⟩  1.0|🔵⟩
          1.0|⭕️⟩

In [40]:
vectorpp(psi[3],dag(linkind(psi,2)),[empty,full])

2-element Vector{ket}:
 1.0|🔵⟩
 1.0|⭕️⟩

In [41]:
norm(psi)^2

2.9999999999999996

In [42]:
site=Index(QN("n",0)=>1,QN("n",1)=>1,tags="site")

(dim=2|id=349|"site") <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1

In [43]:
site1=addtags(site,"s=1")
site2=addtags(site,"s=2")
site3=addtags(site,"s=3")
site4=addtags(site,"s=4")
site5=addtags(site,"s=5")
bond1=Index(QN("n",0)=>1,QN("n",1)=>1,tags="bond1")
bond2=settags(bond1,"bond2")
bond3=settags(bond1,"bond3")
bond4=settags(bond1,"bond4")
A1=onehot(site1=>1,bond1=>1)+onehot(site1=>2,bond1=>2)
A2=ITensor([1 0;0 1],dag(bond1),bond2)*onehot(site2=>1)+ITensor([0 1;0 0],dag(bond1),bond2)*onehot(site2=>2)
A3=ITensor([1 0;0 1],dag(bond2),bond3)*onehot(site3=>1)+ITensor([0 1;0 0],dag(bond2),bond3)*onehot(site3=>2)
A4=ITensor([1 0;0 1],dag(bond3),bond4)*onehot(site4=>1)+ITensor([0 1;0 0],dag(bond3),bond4)*onehot(site4=>2)
A5=onehot(site5=>1,dag(bond4)=>2)+onehot(site5=>2,dag(bond4)=>1)
matrixpp(A4,dag(bond3),bond4,[empty,full])
psi1=MPS([site1,site2,site3,site4,site5])
psi1[1]=A1
psi1[2]=A2
psi1[3]=A3
psi1[4]=A4
psi1[5]=A5
vectorpp(psi1[1],linkind(psi1,1),[empty,full])
matrixpp(psi1[2],dag(linkind(psi1,1)),linkind(psi1,2),[empty,full])
matrixpp(psi1[3],dag(linkind(psi1,2)),linkind(psi1,3),[empty,full])
matrixpp(psi1[4],dag(linkind(psi1,3)),linkind(psi1,4),[empty,full])
vectorpp(psi1[5],dag(linkind(psi1,4)),[empty,full])

2-element Vector{ket}:
 1.0|🔵⟩
 1.0|⭕️⟩

In [44]:
norm(psi1)^2

5.000000000000001

**3 site MPO**

In [45]:
hlink1=Index(QN("n",0)=>1,QN("n",-1)=>1,QN("n",1)=>1,QN("n",0)=>1)
hlink2=settags(hlink1,"hlink2")

(dim=4|id=489|"hlink2") <Out>
 1: QN("n",0) => 1
 2: QN("n",-1) => 1
 3: QN("n",1) => 1
 4: QN("n",0) => 1

In [46]:
ITensors.op(::OpName"id",::SiteType"site")=[1 0;0 1]#defining the identity operator
ITensors.op(::OpName"a",::SiteType"site")=[0 1;0 0]#annhilation operator
ITensors.op(::OpName"ad",::SiteType"site")=[0 0;1 0]#creation operator

In [47]:
H1=op("id",site1)*onehot(hlink1=>1)-op("a",site1)*onehot(hlink1=>2)-op("ad",site1)*onehot(hlink1=>3)

ITensor ord=3
(dim=2|id=349|"s=1,site")' <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
(dim=2|id=349|"s=1,site") <In>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
(dim=4|id=489) <Out>
 1: QN("n",0) => 1
 2: QN("n",-1) => 1
 3: QN("n",1) => 1
 4: QN("n",0) => 1
NDTensors.BlockSparse{Float64, Vector{Float64}, 3}

In [48]:
H2=op("id",site2)*onehot(dag(hlink1)=>1,hlink2=>1)-op("a",site2)*onehot(dag(hlink1)=>1,hlink2=>2)-op("ad",site2)*onehot(dag(hlink1)=>1,hlink2=>3)+
    op("ad",site2)*onehot(dag(hlink1)=>2,hlink2=>4)+op("ad",site2)*onehot(dag(hlink1)=>3,hlink2=>4)+op("ad",site2)*onehot(dag(hlink1)=>4,hlink2=>4)

ITensor ord=4
(dim=2|id=349|"s=2,site")' <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
(dim=2|id=349|"s=2,site") <In>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
(dim=4|id=489) <In>
 1: QN("n",0) => 1
 2: QN("n",-1) => 1
 3: QN("n",1) => 1
 4: QN("n",0) => 1
(dim=4|id=489|"hlink2") <Out>
 1: QN("n",0) => 1
 2: QN("n",-1) => 1
 3: QN("n",1) => 1
 4: QN("n",0) => 1
NDTensors.BlockSparse{Float64, Vector{Float64}, 4}

In [49]:
H3=op("ad",site3)*onehot(dag(hlink2)=>2)+op("a",site3)*onehot(dag(hlink2)=>3)+op("id",site3)*onehot(dag(hlink2)=>4)

ITensor ord=3
(dim=2|id=349|"s=3,site")' <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
(dim=2|id=349|"s=3,site") <In>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
(dim=4|id=489|"hlink2") <In>
 1: QN("n",0) => 1
 2: QN("n",-1) => 1
 3: QN("n",1) => 1
 4: QN("n",0) => 1
NDTensors.BlockSparse{Float64, Vector{Float64}, 3}

In [50]:
@show Htot=H1*H2*H3;

Htot = H1 * H2 * H3 = ITensor ord=6
Dim 1: (dim=2|id=349|"s=1,site")' <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
Dim 2: (dim=2|id=349|"s=1,site") <In>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
Dim 3: (dim=2|id=349|"s=2,site")' <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
Dim 4: (dim=2|id=349|"s=2,site") <In>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
Dim 5: (dim=2|id=349|"s=3,site")' <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
Dim 6: (dim=2|id=349|"s=3,site") <In>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
NDTensors.BlockSparse{Float64, Vector{Float64}, 6}
 2×2×2×2×2×2
Block(1, 1, 1, 2, 2, 1)
 [1:1, 1:1, 1:1, 2:2, 2:2, 1:1]
[:, :, 1, 1, 1, 1] =
 -1.0

Block(1, 1, 2, 1, 1, 2)
 [1:1, 1:1, 2:2, 1:1, 1:1, 2:2]
[:, :, 1, 1, 1, 1] =
 -1.0

Block(2, 2, 1, 2, 2, 1)
 [2:2, 2:2, 1:1, 2:2, 2:2, 1:1]
[:, :, 1, 1, 1, 1] =
 -1.0

Block(2, 2, 2, 1, 1, 2)
 [2:2, 2:2, 2:2, 1:1, 1:1, 2:2]
[:, :, 1, 1, 1, 1] =
 -1.0

Block(1, 2, 2, 1, 1, 1)
 [1:1, 2:2, 2:2, 1:1, 1:1, 1:1]
[:, :, 1, 1, 1, 1] =
 -1.0

Block(1, 2, 2, 1, 

In [51]:
site__1=linkind(psi,1)
psidag_j=dag(prime(psi[1],"site__1"))
res_H1=psidag_j*H1*psi[1]
site__2=linkind(psi,2)
psidag_j=dag(prime(psi[2],"site__2"))
res_H2=psidag_j*H2*psi[2]
site__3=linkind(psi,3)
psidag_j=dag(prime(psi[3],"site__3"))
res_H3=psidag_j*H3*psi[3]

ITensor ord=3
(dim=2|id=349|"s=3,site")' <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
(dim=2|id=349|"s=3,site") <In>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
(dim=4|id=489|"hlink2") <In>
 1: QN("n",0) => 1
 2: QN("n",-1) => 1
 3: QN("n",1) => 1
 4: QN("n",0) => 1
NDTensors.BlockSparse{Float64, Vector{Float64}, 3}

In [63]:
siten=Index(QN("n",0)=>1,QN("n",1)=>1,tags="siten")
siten1=addtags(siten,"s=1")
siten2=addtags(siten,"s=2")
siten3=addtags(siten,"s=3")
bondn1=Index(QN("n",0)=>1,QN("n",1)=>1,tags="bondn1")
bondn2=settags(bondn1,"bondn2")
A1=onehot(siten1=>1,bondn1=>1)+onehot(siten1=>2,bondn1=>2)
A2=ITensor([1 0;0 1],dag(bondn1),bondn2)*onehot(siten2=>1)+ITensor([0 1;0 0],dag(bondn1),bondn2)*onehot(siten2=>2)
A3=onehot(siten3=>1,dag(bondn2)=>2)+onehot(siten3=>2,dag(bondn2)=>1)
psin=MPS([siten1,siten2,siten3])
psin[1]=A1
psin[2]=A2
psin[3]=A3
hlinkn1=Index(QN("n",0)=>1,QN("n",-1)=>1,QN("n",1)=>1,QN("n",0)=>1,tags="hlinkn1")
hlinkn2=settags(hlinkn1,"hlinkn2")
ITensors.op(::OpName"id",::SiteType"siten")=[1 0;0 1]
ITensors.op(::OpName"a",::SiteType"siten")=[0 1;0 0]
ITensors.op(::OpName"ad",::SiteType"siten")=[0 0;1 0]
H1=op("id",siten1)*onehot(hlinkn1=>1)-op("a",siten1)*onehot(hlinkn1=>2)-op("ad",siten1)*onehot(hlinkn1=>3)
H2=op("id",siten2)*onehot(dag(hlinkn1)=>1,hlinkn2=>1)-op("a",siten2)*onehot(dag(hlinkn1)=>1,hlinkn2=>2)-op("ad",siten2)*onehot(dag(hlinkn1)=>1,hlinkn2=>3)+
   op("ad",siten2)*onehot(dag(hlinkn1)=>2,hlinkn2=>4)+op("a",siten2)*onehot(dag(hlinkn1)=>3,hlinkn2=>4)+op("id",siten2)*onehot(dag(hlinkn1)=>4,hlinkn2=>4)
H3=op("ad",siten3)*onehot(dag(hlinkn2)=>2)+op("a",siten3)*onehot(dag(hlinkn2)=>3)+op("id",siten3)*onehot(dag(hlinkn2)=>4)
Hmpon=MPO([site1,site2,site3])
Hmpon[1]=H1
Hmpon[2]=H2
Hmpon[3]=H3
inner(psin',Hmpon,psin)

-4.0

In [64]:
res_H1,res_H2,res_H3

(ITensor ord=3
Dim 1: (dim=2|id=349|"s=1,site")' <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
Dim 2: (dim=2|id=349|"s=1,site") <In>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
Dim 3: (dim=4|id=489) <Out>
 1: QN("n",0) => 1
 2: QN("n",-1) => 1
 3: QN("n",1) => 1
 4: QN("n",0) => 1
NDTensors.BlockSparse{Float64, Vector{Float64}, 3}
 2×2×4
Block(1, 1, 1)
 [1:1, 1:1, 1:1]
[:, :, 1] =
 2.0

Block(2, 2, 1)
 [2:2, 2:2, 1:1]
[:, :, 1] =
 2.0

Block(1, 2, 2)
 [1:1, 2:2, 2:2]
[:, :, 1] =
 -2.0

Block(2, 1, 3)
 [2:2, 1:1, 3:3]
[:, :, 1] =
 -2.0, ITensor ord=4
Dim 1: (dim=2|id=349|"s=2,site")' <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
Dim 2: (dim=2|id=349|"s=2,site") <In>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
Dim 3: (dim=4|id=489) <In>
 1: QN("n",0) => 1
 2: QN("n",-1) => 1
 3: QN("n",1) => 1
 4: QN("n",0) => 1
Dim 4: (dim=4|id=489|"hlink2") <Out>
 1: QN("n",0) => 1
 2: QN("n",-1) => 1
 3: QN("n",1) => 1
 4: QN("n",0) => 1
NDTensors.BlockSparse{Float64, Vector{Float64}, 4}
 2×2×4×4
Block(1, 1, 1, 1)
 [

In [65]:
site=Index(QN("n",0)=>1,QN("n",1)=>1,tags="site")
site1=addtags(site,"s=1")
site2=addtags(site,"s=2")
site3=addtags(site,"s=3")
bond1=Index(QN("n",0)=>1,QN("n",1)=>1,tags="bond1")
bond2=settags(bond1,"bond2")
A1=onehot(site1=>1,bond1=>1)+onehot(site1=>2,bond1=>2)
A2=ITensor([1 0;0 1],dag(bond1),bond2)*onehot(site2=>1)+ITensor([0 1;0 0],dag(bond1),bond2)*onehot(site2=>2)
A3=onehot(site3=>1,dag(bond2)=>2)+onehot(site3=>2,dag(bond2)=>1)
psi=MPS([site1,site2,site3])
psi[1]=A1
psi[2]=A2
psi[3]=A3


ITensor ord=2
(dim=2|id=563|"s=3,site") <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
(dim=2|id=994|"bond2") <In>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
NDTensors.BlockSparse{Float64, Vector{Float64}, 2}

In [66]:
# create link indices
hlink1=Index(QN("n",0)=>1,QN("n",-1)=>1,QN("n",1)=>1,QN("n",0)=>1,tags="hlink1")
hlink2=settags(hlink1,"hlink2")
# create operators for identity, annihilation and creation operators
ITensors.op(::OpName"id",::SiteType"site")=[1 0;0 1]
ITensors.op(::OpName"a",::SiteType"site")=[0 1;0 0]
ITensors.op(::OpName"ad",::SiteType"site")=[0 0;1 0]
H1=op("id",site1)*onehot(hlink1=>1)-op("a",site1)*onehot(hlink1=>2)-op("ad",site1)*onehot(hlink1=>3)
H2=op("id",site2)*onehot(dag(hlink1)=>1,hlink2=>1)-op("a",site2)*onehot(dag(hlink1)=>1,hlink2=>2)-
    op("ad",site2)*onehot(dag(hlink1)=>1,hlink2=>3)+op("ad",site2)*onehot(dag(hlink1)=>2,hlink2=>4)+
    op("a",site2)*onehot(dag(hlink1)=>3,hlink2=>4)+op("id",site2)*onehot(dag(hlink1)=>4,hlink2=>4)
H3=op("id",site3)*onehot(dag(hlink2)=>4)+op("a",site3)*onehot(dag(hlink2)=>3)+op("ad",site3)*onehot(dag(hlink2)=>2);
@show H3

H3 = ITensor ord=3
Dim 1: (dim=2|id=563|"s=3,site")' <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
Dim 2: (dim=2|id=563|"s=3,site") <In>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
Dim 3: (dim=4|id=40|"hlink2") <In>
 1: QN("n",0) => 1
 2: QN("n",-1) => 1
 3: QN("n",1) => 1
 4: QN("n",0) => 1
NDTensors.BlockSparse{Float64, Vector{Float64}, 3}
 2×2×4
Block(1, 1, 4)
 [1:1, 1:1, 4:4]
[:, :, 1] =
 1.0

Block(2, 2, 4)
 [2:2, 2:2, 4:4]
[:, :, 1] =
 1.0

Block(1, 2, 3)
 [1:1, 2:2, 3:3]
[:, :, 1] =
 1.0

Block(2, 1, 2)
 [2:2, 1:1, 2:2]
[:, :, 1] =
 1.0


ITensor ord=3
(dim=2|id=563|"s=3,site")' <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
(dim=2|id=563|"s=3,site") <In>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
(dim=4|id=40|"hlink2") <In>
 1: QN("n",0) => 1
 2: QN("n",-1) => 1
 3: QN("n",1) => 1
 4: QN("n",0) => 1
NDTensors.BlockSparse{Float64, Vector{Float64}, 3}

**Data structures to store the MPOs**

In [67]:
Hmpo=MPO([site1,site2,site3])
Hmpo[1]=H1
Hmpo[2]=H2
Hmpo[3]=H3;

In [68]:
inner(psi',Hmpo,psi)

-4.0

In [69]:
hterms=OpSum()
hterms += (-1,"a",1,"ad",2) # - a1 a2^dag
hterms += (-1,"ad",1,"a",2) # - a1^dag a2
hterms += (-1,"a",2,"ad",3) # - a2 a3^dag
hterms += (-1,"ad",2,"a",3) # - a2^dag a3
altHmpo=MPO(hterms,[site1,site2,site3]);
inner(psi',altHmpo,psi)

-4.0

In [70]:
altHmpo[1]

ITensor ord=3
(dim=4|id=713|"Link,l=1") <Out>
 1: QN() => 1
 2: QN("n",1) => 1
 3: QN("n",-1) => 1
 4: QN("n",0) => 1
(dim=2|id=563|"s=1,site")' <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
(dim=2|id=563|"s=1,site") <In>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
NDTensors.BlockSparse{Float64, Vector{Float64}, 3}

In [71]:
array(altHmpo[1])[1,:,:]

2×2 Matrix{Float64}:
 0.0  0.0
 0.0  0.0

In [72]:
array(altHmpo[1])[2,:,:]

2×2 Matrix{Float64}:
 0.0  -1.0
 0.0   0.0

In [73]:
array(altHmpo[1])[3,:,:]

2×2 Matrix{Float64}:
 0.0  0.0
 1.0  0.0

In [74]:
array(altHmpo[1])[4,:,:]

2×2 Matrix{Float64}:
 1.0  0.0
 0.0  1.0

In [75]:
linkinds(altHmpo)

2-element Vector{Index{Vector{Pair{QN, Int64}}}}:
 (dim=4|id=713|"Link,l=1") <Out>
 1: QN() => 1
 2: QN("n",1) => 1
 3: QN("n",-1) => 1
 4: QN("n",0) => 1
 (dim=4|id=454|"Link,l=2") <Out>
 1: QN() => 1
 2: QN("n",1) => 1
 3: QN("n",-1) => 1
 4: QN("n",0) => 1

In [76]:
site=Index(QN("n",0)=>1,QN("n",1)=>1,tags="site")
site1=addtags(site,"s=1")
site2=addtags(site,"s=2")
site3=addtags(site,"s=3")
site=[site1,site2,site3]
ITensors.op(::OpName"Num",::SiteType"site")=[0 1;0 0]
ITensors.op(::OpName"a",::SiteType"site")=[0 1;0 0]
ITensors.op(::OpName"ad",::SiteType"site")=[0 0;1 0]
hterms=OpSum()
hterms+=(-1,"a",1,"ad",2)
hterms+=(-1,"ad",1,"a",2)
hterms+=(-1,"a",2,"ad",3)
hterms+=(-1,"ad",2,"a",3)
Hmpo=MPO(hterms,[site1,site2,site3]);
psi[1]=onehot(site1=>1,bond1=>1)+onehot(site1=>2,bond1=>2)
psi[2]=ITensor([1 0;0 1],dag(bond1),bond2)*onehot(site2=>1)+ITensor([0 1;0 0],dag(bond1),bond2)*onehot(site2=>2)
psi[3]=onehot(dag(bond2)=>1,site3=>1)+onehot(dag(bond2)=>2,site3=>2)
# vectorpp(psi[3],dag(linkind(psi,2)),[empty,full])

ITensor ord=2
(dim=2|id=994|"bond2") <In>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
(dim=2|id=382|"s=3,site") <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
NDTensors.BlockSparse{Float64, Vector{Float64}, 2}

In [77]:
op("ad",site2)

ITensor ord=2
(dim=2|id=382|"s=2,site")' <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
(dim=2|id=382|"s=2,site") <In>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
NDTensors.BlockSparse{Float64, Vector{Float64}, 2}

In [78]:
expect(psi,"Num")

LoadError: Fluxes not all equal

In [79]:
op("Num",site1)

ITensor ord=2
(dim=2|id=382|"s=1,site")' <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
(dim=2|id=382|"s=1,site") <In>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
NDTensors.BlockSparse{Float64, Vector{Float64}, 2}

In [80]:
energy1,psi1=dmrg(Hmpo,psi;nsweeps=1,maxdim=2,cutoff=0)

LoadError: Fluxes not all equal

In [81]:
n1=expect(psi1,"Num")

LoadError: Fluxes not all equal

**calculating for n-sites**

In [82]:
sitem = Index(QN("n", 0) => 1, QN("n", 1) => 1, tags="sitem")
bond1 = Index(QN("n", 0) => 1, QN("n", 1) => 1, tags="bond1")
n = 3
mps_tensor=Vector{ITensor}(undef,n)
siten = Vector{Index}(undef, n)
bondn=Vector{Index}(undef,n)
for i in 1:n
     siten[i]=addtags(sitem,"s=$i")
end
# c=addtags(ITensor(siten[1]),"s=1") #trial run
mps_tensors = Vector{ITensor}(undef, n)
for i in 1:n
    mps_tensors[i] = addtags(ITensor(siten[i]), "s=$i")
end
bondn[1]=bond1
for i in 2:n-1
    bondn[i]=settags(bondn[1],"bond$i")
end
psi_n=MPS(mps_tensors);
psi_n[1]=onehot(siten[1]=>1,bondn[1]=>1)+onehot(siten[1]=>2,bondn[1]=>2)
psi_n[n]=onehot(siten[n]=>1,dag(bondn[n-1])=>2)+onehot(siten[n]=>2,dag(bondn[n-1])=>1)
for i in 2:n-1
    psi_n[i]=ITensor([1 0;0 1],dag(bondn[i-1]),bondn[i])*onehot(siten[i]=>1)+
             ITensor([0 1;0 0],dag(bondn[i-1]),bondn[i])*onehot(siten[i]=>2)
end
#making the matrix product operator for n sites
ITensors.op(::OpName"a",::SiteType"sitem")=[0 1;0 0]
ITensors.op(::OpName"ad",::SiteType"sitem")=[0 0;1 0]
hterms=OpSum()
for i in 1:n-1
            hterms+=(-1,"a",i,"ad",i+1)
            hterms+=(-1,"ad",i,"a",i+1)
            # hterms+=(-1,"a",i+1,"ad",i+2)
            # hterms+=(-1,"ad",i+1,"a",i+2)
end
Hmpon=MPO(hterms,siten);

In [83]:
inner(psi_n',Hmpon,psi_n)

-4.0

In [84]:
ITensors.op(::OpName"Num",::SiteType"sitem")=[0 0;0 1]

In [85]:
energy1,psi1=dmrg(Hmpon,psi_n,nsweeps=1,maxdim=2,cutoff=0)

LoadError: Fluxes not all equal

In [86]:
A1=psi_n[2]
matrixpp(A1,dag(bondn[1]),bondn[2],[empty,full])

2×2 Matrix{ket}:
 1.0|⭕️⟩  1.0|🔵⟩
          1.0|⭕️⟩

In [87]:
sitem = Index(QN("n", 0) => 1, QN("n", 1) => 1, tags="sitem")
bond1 = Index(QN("n", 0) => 1, QN("n", 1) => 1, tags="bond1")
n = 3
mps_tensor=Vector{ITensor}(undef,n)
siten = Vector{Index}(undef, n)
bondn=Vector{Index}(undef,n)
for i in 1:n
     siten[i]=addtags(sitem,"s=$i")
end
# c=addtags(ITensor(siten[1]),"s=1") #trial run
mps_tensors = Vector{ITensor}(undef, n)
for i in 1:n
    mps_tensors[i] = addtags(ITensor(siten[i]), "s=$i")
end
bondn[1]=bond1
for i in 2:n-1
    bondn[i]=settags(bondn[1],"bond$i")
end
psi_n=MPS(mps_tensors);
psi_n[1]=onehot(siten[1]=>1,bondn[1]=>2)+onehot(siten[1]=>1,bondn[1]=>1)
psi_n[n]=onehot(siten[n]=>1,dag(bondn[n-1])=>1)+onehot(siten[n]=>1,dag(bondn[n-1])=>2)
for i in 2:n-2
    psi_n[i]=ITensor([1 0;0 1],dag(bondn[i-1]),bondn[i])*onehot(siten[i]=>1)+
             ITensor([0 1;0 0],dag(bondn[i-1]),bondn[i])*onehot(siten[i]=>1)
end
psi_n[n-1]=ITensor([1 0;0 1],dag(bondn[n-2]),bondn[n-1])*onehot(siten[n-1]=>1)+
           ITensor([0 1;0 0],dag(bondn[n-2]),bondn[n-1])*onehot(siten[n-1]=>2)
matrixpp(psi_n[n-1],dag(bondn[n-2]),bondn[n-1],[empty,full])

2×2 Matrix{ket}:
 1.0|⭕️⟩  1.0|🔵⟩
          1.0|⭕️⟩

In [88]:
matrixpp(psi_n[n-3],dag(bondn[n-4]),bondn[n-3],[empty,full])

LoadError: BoundsError: attempt to access 3-element Vector{ITensor} at index [0]

In [89]:
vectorpp(psi_n[1],bondn[1],[empty,full])

2-element Vector{ket}:
 1.0|⭕️⟩
 1.0|⭕️⟩

In [90]:
vectorpp(psi_n[n],dag(bondn[n-1]),[empty,full])

2-element Vector{ket}:
 1.0|⭕️⟩
 1.0|⭕️⟩

In [91]:
sitem1=addtags(sitem,"sitem=1");
sitem2=addtags(sitem,"sitem=2");
bon_d1=Index(QN("n",0)=>1,QN("n",1)=>1,QN("n",-1)=>1,tags="bon_d1");
psi_nm=MPS([sitem1,sitem2]);
A_1=onehot(sitem1=>1,bon_d1=>1)+onehot(sitem1=>2,bon_d1=>2)
A_2=onehot(dag(bon_d1)=>1,sitem2=>2)+onehot(dag(bon_d1)=>2,sitem2=>1)
vectorpp(A_2,dag(bon_d1),[empty,full])
psi_nm[1]=A_1;
psi_nm[2]=A_2;

In [92]:
psi_nm

MPS
[1] ((dim=2|id=852|"sitem,sitem=1") <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1, (dim=3|id=838|"bon_d1") <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
 3: QN("n",-1) => 1)
[2] ((dim=3|id=838|"bon_d1") <In>
 1: QN("n",0) => 1
 2: QN("n",1) => 1
 3: QN("n",-1) => 1, (dim=2|id=852|"sitem,sitem=2") <Out>
 1: QN("n",0) => 1
 2: QN("n",1) => 1)


In [93]:
import Pkg; 
Pkg.add("ITensorMPS")

    Updating registry at `C:\Users\shikh\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\shikh\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\shikh\.julia\environments\v1.10\Manifest.toml`


In [94]:
psi_n=MPS(mps_tensors);
psi_n[1]=onehot(siten[1]=>1,bondn[1]=>1)+onehot(siten[1]=>2,bondn[1]=>2)
psi_n[n]=onehot(siten[n]=>1,dag(bondn[n-1])=>2)+onehot(siten[n]=>2,dag(bondn[n-1])=>1)
for i in 2:n-1
    psi_n[i]=ITensor([1 0;0 1],dag(bondn[i-1]),bondn[i])*onehot(siten[i]=>1)+
             ITensor([0 1;0 0],dag(bondn[i-1]),bondn[i])*onehot(siten[i]=>2)
end

In [95]:
import Pkg
Pkg.add("ITensorMPS")
Pkg.add("ITensors")

   Resolving package versions...
  No Changes to `C:\Users\shikh\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\shikh\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\shikh\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\shikh\.julia\environments\v1.10\Manifest.toml`


In [96]:
using ITensors,ITensorMPS
let
    N=5
    # sitesn = siteinds(n->iseven(n) ? "S=1" : "S=1",2)
    sitesm = siteinds(n->iseven(n) ? "S=1" : "S=1",N)
    # Jn=1
    # ITensors.op(::OpName"a",::SiteType"sitesn[1]")
    os=OpSum()
    for i in 1:N-1
        os+=(-1,"S-",i,"S+",i+1)
        os+=(-1,"S+",i,"S-",i+1)
    end
    H=MPO(os,sitesm)
    nsweeps=10
    maxdim=[10,20,30,40,50,60]
    cutoff=[1E-8]
    psi=random_mps(sitesm;linkdims=2)
    psi=normalize(psi)
    energy,psin=dmrg(H,psi;nsweeps,maxdim,cutoff)
    return 
end

After sweep 1 energy=-9.281906188439965  maxlinkdim=9 maxerr=6.98E-17 time=18.206
After sweep 2 energy=-9.35822766834848  maxlinkdim=9 maxerr=2.54E-16 time=0.004
After sweep 3 energy=-9.362049681566363  maxlinkdim=9 maxerr=2.15E-16 time=0.004
After sweep 4 energy=-9.362108335281679  maxlinkdim=9 maxerr=1.13E-16 time=0.004
After sweep 5 energy=-9.362108946657115  maxlinkdim=9 maxerr=9.72E-17 time=0.004
After sweep 6 energy=-9.362108954142242  maxlinkdim=9 maxerr=1.45E-16 time=0.006
After sweep 7 energy=-9.362108954236303  maxlinkdim=9 maxerr=0.00E+00 time=0.007
After sweep 8 energy=-9.36210895423682  maxlinkdim=9 maxerr=9.06E-17 time=0.039
After sweep 9 energy=-9.36210895423682  maxlinkdim=9 maxerr=1.20E-16 time=0.004
After sweep 10 energy=-9.362108954236824  maxlinkdim=9 maxerr=1.93E-16 time=0.004


In [97]:
using ITensors,ITensorMPS
let
    N=2
    # sitesn = siteinds(n->iseven(n) ? "S=1" : "S=1",2)
    sitesm = siteinds(n->iseven(n) ? "S=1" : "S=1",N)
    # Jn=1
    # ITensors.op(::OpName"a",::SiteType"sitesn[1]")
    os=OpSum()
    for i in 1:N-1
        os+=(-1,"S-",i,"S+",i+1)
        os+=(-1,"S+",i,"S-",i+1)
    end
    H=MPO(os,sitesm)
    nsweeps=10
    maxdim=[10,20,30,40,50,60]
    cutoff=[1E-8]
    psi=random_mps(sitesm;linkdims=3)
    psi=normalize(psi)
    energy,psin=dmrg(H,psi;nsweeps,maxdim,cutoff)
    return 
end

After sweep 1 energy=-2.8110660166206394  maxlinkdim=3 maxerr=0.00E+00 time=2.268
After sweep 2 energy=-2.8284199606541165  maxlinkdim=3 maxerr=0.00E+00 time=0.001
After sweep 3 energy=-2.828427114267681  maxlinkdim=3 maxerr=0.00E+00 time=0.001
After sweep 4 energy=-2.8284271247300055  maxlinkdim=3 maxerr=0.00E+00 time=0.001
After sweep 5 energy=-2.828427124746165  maxlinkdim=3 maxerr=0.00E+00 time=0.001
After sweep 6 energy=-2.8284271247461907  maxlinkdim=3 maxerr=0.00E+00 time=0.001
After sweep 7 energy=-2.8284271247461907  maxlinkdim=3 maxerr=0.00E+00 time=0.001
After sweep 8 energy=-2.8284271247461903  maxlinkdim=3 maxerr=0.00E+00 time=0.001
After sweep 9 energy=-2.8284271247461903  maxlinkdim=3 maxerr=0.00E+00 time=0.001
After sweep 10 energy=-2.8284271247461907  maxlinkdim=3 maxerr=0.00E+00 time=0.001


In [98]:
?prod

search: prod prod! Prod product productMPS cumprod cumprod! nextprod



```
prod(f, itr; [init])
```

Return the product of `f` applied to each element of `itr`.

The return type is `Int` for signed integers of less than system word size, and `UInt` for unsigned integers of less than system word size.  For all other arguments, a common return type is found to which all arguments are promoted.

The value returned for empty `itr` can be specified by `init`. It must be the multiplicative identity (i.e. one) as it is unspecified whether `init` is used for non-empty collections.

!!! compat "Julia 1.6"
    Keyword argument `init` requires Julia 1.6 or later.


# Examples

```jldoctest
julia> prod(abs2, [2; 3; 4])
576
```

---

```
prod(itr; [init])
```

Return the product of all elements of a collection.

The return type is `Int` for signed integers of less than system word size, and `UInt` for unsigned integers of less than system word size.  For all other arguments, a common return type is found to which all arguments are promoted.

The value returned for empty `itr` can be specified by `init`. It must be the multiplicative identity (i.e. one) as it is unspecified whether `init` is used for non-empty collections.

!!! compat "Julia 1.6"
    Keyword argument `init` requires Julia 1.6 or later.


See also: [`reduce`](@ref), [`cumprod`](@ref), [`any`](@ref).

# Examples

```jldoctest
julia> prod(1:5)
120

julia> prod(1:5; init = 1.0)
120.0
```

---

```
prod(A::AbstractArray; dims)
```

Multiply elements of an array over the given dimensions.

# Examples

```jldoctest
julia> A = [1 2; 3 4]
2×2 Matrix{Int64}:
 1  2
 3  4

julia> prod(A, dims=1)
1×2 Matrix{Int64}:
 3  8

julia> prod(A, dims=2)
2×1 Matrix{Int64}:
  2
 12
```

---

```
prod(f, A::AbstractArray; dims)
```

Multiply the results of calling the function `f` on each element of an array over the given dimensions.

# Examples

```jldoctest
julia> A = [1 2; 3 4]
2×2 Matrix{Int64}:
 1  2
 3  4

julia> prod(abs2, A, dims=1)
1×2 Matrix{Int64}:
 9  64

julia> prod(abs2, A, dims=2)
2×1 Matrix{Int64}:
   4
 144
```


In [99]:
function ncon_tensordot(A,B,Acont,Bcont)
    Afree=deleteat!(collect(1:ndims(A)),sort(Acont))
    Bfree=deleteat!(collect(1:ndims(B)),sort(Bcont))
    Aperm=vcat(Afree,Acont)
    Bperm=vcat(Bcont,Bfree)
    return reshape(
        reshape(
            permutedims(A,Aperm),
            prod(size(A)[Afree]),
            prod(size(A)[Acont]),
            )*rehsape(
            permutedims(B,Bperm),
            prod(size(B)[Bcont]),
            prod(size(B)[Bfree]),
            ),
        (size(A)[Afree]...,size(B)[free]...),
    )
end
function ncon_partial_trace(A,Alabel)
    num_count=length(Alabel)-length(unique(Alabel))
    if num_count>0
        dup_list=[]
        for ele in unique(Alabel)
            if sum(Albel .==ele)>1
                dup_list=vcat(dup_list,finall(Alabel .==ele))
            end
        end
        cont_ind=reshape(permutedims(reshape(dup_list,2,num_count),[2,1]),2*num_count)
        free_ind=deleteat!(collect(1:length(Alabel)),sort(dup_list))
        cont_dim=prod(size(A),[cont_ind[1:num_count]])
        free_dim=prod(size(A)[free_ind])
        cont_label=unique(Alabel[cont_ind])
        B=zeros(prod(free_dim))
        Adims=size(A)
        A=rehshape(permutedims(reshape(A,Adims),vcat(free_ind.cont_ind)),prod(free_dim),cont_ind,cont_ind)
        for ip =1:cont_dim
            B=B+A[:,ip,ip]
        end
        return reshape(B,free_dim),deleteat!(Alabel,sort(cont_ind)),cont_label
    else
        return A,Alabel
    end
end

ncon_partial_trace (generic function with 1 method)

In [100]:
function con_partial_trace(A,Alabel)
    num_cont=length(Alabel)-length(unique(Alabel))
    if num_cont>0
        dup_list=[]
        for ele in unique(Alabel)
            if sum(Alabel.==ele)>1
                dup_list=vcat(dup_list,findall(Alabel.==ele))
            end
        end
        cont_ind=reshape(permutedims(reshape(dup_list,2,num_count),[2,1]),2*num_cont)
        free_ind=deleteat!(collect(1:length(Alabel)),sort(dup_lsit))
        cont_dim=size(A)[cont_ind[1:num_cont]]
        free_dim=size(A)[free_ind]
        cont_label=unique(Alabel[cont_ind])
        B=zeros(prod(free_dim))
        Adims=size(A)
        A=reshape(
            permutedims(reshape(A[:],Adims),vcat(free_ind,cont_ind)),
            prod(free_dim),
            cont_dim,
            cont_dim,
        )
        for ip in 1:cont_dim
            B=B+A[:, ip, ip]
        end
        return reshape(B,free_dim),deleteat!(Alabel,sort(cont_ind)),cont_label
    else
        return A,Alabel
    end
end

con_partial_trace (generic function with 1 method)

In [101]:
function ncon_check_inputs(connect_list,flat_connect,dims_list,con_order)
    pos_ind=flat_connect[flat_connect .>0]
    neg_ind=flat_connect[flat_connect .<0]
    if length(dims_list)!=length(connect_list)
        e_str0 = "NCON error: $(length(dims_list)) tensors given ";
        error(e_str0,"but $(length(connect_list)) index sublists given")
    end
    for ik=1:length(dims_list)
        if length(dims_list[ik])!=length(connect_list[ik])
            e_str0 = "number of indices does not match number of labels on tensor ";
            e_str1 = "$(ik): $(length(dims_list[ik]))-indices "
            error(e_str0,e_str1,"versus $(length(connect_list[ik]))-labels")
        end
    end
    if !(sort(con_order)==sort(unique(pos_ind)))
        error("NCON error : invalid contraction order")
    end
    #checking for the validity of the negative inds
    for ind = -1:-1:length(neg_ind)
        if sum(neg_ind .== ind) == 0
            error("NCON error:no index labelled $(ind)")
        elseif sum(neg_ind .==ind) > 1
            error("NCON error more than one index labelled")
        end
    end
    #check positive indices fro validity and contracted tensor dimensions match
    flat_dims=[]
    for ele in dims_list
        flat_dims=vcat(flat_dims,ele)
    end
    for ind in unique(pos_ind)
        if sum(pos_ind .==ind)==1
            error("NCON error: only one index labelled $(ind)")
        elseif sum(pos_ind .==ind)>2
            error("NCON error : more than one index labelled $(ind)")
        end
        cont_dims= flat_dims[flat_connect .==ind]
        if cont_dims[1] !=cont_dims[2]
            e_str0 ="dimension mismatch on index labelled $(ind):"
            error(e_str0,"dim-$(cont_dims[1]) versus dim-$(cont_dims[2])")
        end
    end
    return true
end

ncon_check_inputs (generic function with 1 method)

In [102]:
a=[1,1,2,3,5,5,8,8,9,9,9,6,7,4,3,3]
unique(a)
listu=[]
for ele in unique(a)
    if sum(a .==ele)>1
        listu=vcat(listu,findall(a .==ele))
    end
end
num=length(a)-length(unique(a))
@show reshape(listu,2,6)

reshape(listu, 2, 6) = Any[1 4 16 6 8 10; 2 15 5 7 9 11]


2×6 Matrix{Any}:
 1   4  16  6  8  10
 2  15   5  7  9  11

In [103]:
a1 = [-1, -6, -5, -5, -6, -6, -6, -8, -3, -9]
neg_ind = a1[a1 .< 0]

# Get unique negative numbers
unique_neg = unique(neg_ind)

# Iterate over each unique negative number and print the count
for ind in unique_neg
    println("Number $ind appears $(sum(neg_ind .== ind)) times")
end


Number -1 appears 1 times
Number -6 appears 4 times
Number -5 appears 2 times
Number -8 appears 1 times
Number -3 appears 1 times
Number -9 appears 1 times


In [14]:
function ncon(
  tensor_list,
  connect_list_in;
  con_order = [],
  check_network = true,
)

  # copy original list to avoid destroying
  connect_list = deepcopy(connect_list_in)

  # put inputs into an array if necessary
  if (tensor_list[1] isa Real) | (tensor_list[1] isa Complex)
    tensor_list = Any[tensor_list]
  end
  if !(connect_list[1] isa Array)
    connect_list = Any[connect_list]
  end

  # generate contraction order if necessary
  flat_connect = vcat(connect_list...)
  if isempty(con_order)
    con_order = sort(unique(flat_connect[flat_connect.>0]))
  end

  # check inputs if enabled
  if check_network
    dims_list = Array{Any,1}(undef, length(tensor_list))
    for ik = 1:length(tensor_list)
      dims_list[ik] = [size(tensor_list[ik])...]
    end
    ncon_check_inputs(connect_list, flat_connect, dims_list, con_order)
  end

  # do all partial traces
  for ip = 1:length(connect_list)
    num_cont = length(connect_list[ip]) - length(unique(connect_list[ip]))
    if num_cont > 0
      tensor_list[ip], connect_list[ip], cont_label =
        ncon_partial_trace(tensor_list[ip], connect_list[ip])
      con_order = setdiff(con_order, cont_label)
    end
  end

  # do all binary contractions
  while !isempty(con_order)
    # identify tensors to be contracted
    cont_ind = con_order[1];
    locs = [
      ele
      for
      ele in collect(1:length(connect_list)) if
      sum(connect_list[ele] .== cont_ind) > 0
    ]

    # do a binary contraction
    cont_many = intersect(connect_list[locs[1]], connect_list[locs[2]])
    A_cont = [findfirst(connect_list[locs[1]] .== x) for x in cont_many]
    B_cont = [findfirst(connect_list[locs[2]] .== x) for x in cont_many]
    A_free = deleteat!(collect(1:length(connect_list[locs[1]])), sort(A_cont))
    B_free = deleteat!(collect(1:length(connect_list[locs[2]])), sort(B_cont))
    push!(
      tensor_list,
      ncon_tensordot(
        tensor_list[locs[1]],
        tensor_list[locs[2]],
        A_cont,
        B_cont,
      ),
    )
    push!(
      connect_list,
      vcat(connect_list[locs[1]][A_free], connect_list[locs[2]][B_free]),
    )

    # remove contracted tensors from list and update con_order
    deleteat!(connect_list, locs)
    deleteat!(tensor_list, locs)
    con_order = setdiff(con_order, cont_many)
  end

  # do all outer products
  while length(tensor_list) > 1
    s1 = size(tensor_list[end-1])
    s2 = size(tensor_list[end])
    tensor_list[end-1] = reshape(
      reshape(tensor_list[end-1], prod(s1)) *
      reshape(tensor_list[end], 1, prod(s2)),
      (s1..., s2...),
    )
    connect_list[end-1] = vcat(connect_list[end-1], connect_list[end])
    deleteat!(connect_list, length(connect_list))
    deleteat!(tensor_list, length(tensor_list))
  end

  # do final permutation
  if length(connect_list[1]) > 0
    return permutedims(tensor_list[1], sortperm(connect_list[1], by = abs))
  else
    return tensor_list[1][1]
  end
end

"""
ncon_tensordot: contracts a pair of tensors via matrix multiplication,
similar to the Numpy function of the same name
"""
function ncon_tensordot(A, B, A_cont, B_cont)

  A_free = deleteat!(collect(1:ndims(A)), sort(A_cont))
  B_free = deleteat!(collect(1:ndims(B)), sort(B_cont))
  A_perm = vcat(A_free, A_cont)
  B_perm = vcat(B_cont, B_free)

  return reshape(
    reshape(
      permutedims(A, A_perm),
      prod(size(A)[A_free]),
      prod(size(A)[A_cont]),
    ) * reshape(
      permutedims(B, B_perm),
      prod(size(B)[B_cont]),
      prod(size(B)[B_free]),
    ),
    (size(A)[A_free]..., size(B)[B_free]...),
  )
end

"""
ncon_partial_trace: partial trace on tensor A over repeated labels in A_label
"""
function ncon_partial_trace(A, A_label)

  num_cont = length(A_label) - length(unique(A_label))
  if num_cont > 0
    dup_list = []
    for ele in unique(A_label)
      if sum(A_label .== ele) > 1
        dup_list = vcat(dup_list, findall(A_label .== ele))
      end
    end

    cont_ind =
      reshape(permutedims(reshape(dup_list, 2, num_cont), [2, 1]), 2 * num_cont)
    free_ind = deleteat!(collect(1:length(A_label)), sort(dup_list))
    cont_dim = prod(size(A)[cont_ind[1:num_cont]])
    free_dim = size(A)[free_ind]

    cont_label = unique(A_label[cont_ind])
    B = zeros(prod(free_dim))
    perm_tot = [free_ind; cont_ind]

    A_dims = size(A)
    A = reshape(
      permutedims(reshape(A[:], A_dims), vcat(free_ind, cont_ind)),
      prod(free_dim),
      cont_dim,
      cont_dim,
    )
    for ip = 1:cont_dim
      B = B + A[:, ip, ip]
    end

    return reshape(B, free_dim), deleteat!(A_label, sort(cont_ind)), cont_label
  else
    return A, A_label
  end
end

"""
ncon_check_inputs: check consistency of input tensor network
"""
function ncon_check_inputs(connect_list, flat_connect, dims_list, con_order)

  pos_ind = flat_connect[flat_connect.>0]
  neg_ind = flat_connect[flat_connect.<0]

  # check that lengths of lists match
  if length(dims_list) != length(connect_list)
    e_str0 = "NCON error: $(length(dims_list)) tensors given ";
    error(e_str0,"but $(length(connect_list)) index sublists given")
  end

  # check that tensors have the right number of indices
  for ik = 1:length(dims_list)
    if length(dims_list[ik]) != length(connect_list[ik])
      e_str0 = "number of indices does not match number of labels on tensor ";
      e_str1 = "$(ik): $(length(dims_list[ik]))-indices "
      error(e_str0,e_str1,"versus $(length(connect_list[ik]))-labels")
    end
  end

  # check that contraction order is valid
  if !(sort(con_order) == sort(unique(pos_ind)))
    error("NCON error: invalid contraction order")
  end

  # check that negative indices are valid
  for ind = -1:-1:-length(neg_ind)
    if sum(neg_ind .== ind) == 0
      error("NCON error: no index labelled $(ind)")
    elseif sum(neg_ind .== ind) > 1
      error("NCON error: more than one index labelled $(ind)")
    end
  end

  # check that positive indices are valid and contracted tensor dimensions match
  flat_dims = []
  for ele in dims_list
    flat_dims = vcat(flat_dims, ele)
  end
  for ind in unique(pos_ind)
    if sum(pos_ind .== ind) == 1
      error("NCON error: only one index labelled $(ind)")
    elseif sum(pos_ind .== ind) > 2
      error("NCON error: more than two indices labelled $(ind)")
    end
    cont_dims = flat_dims[flat_connect.==ind]
    if cont_dims[1] != cont_dims[2]
      e_str0 = "dimension mismatch on index labelled $(ind): "
      error(e_str0,"dim-$(cont_dims[1]) versus dim-$(cont_dims[2])")
    end
  end

  return true
end


ncon_check_inputs

# **TEBD algorithm practice**

In [105]:
using LinearAlgebra,Printf
#initializing
chi=10  #bond dimension
d=2  #dimension associated to each site
A=rand(chi,d,chi)
B=rand(chi,d,chi)
sAB=ones(chi) ./sqrt(chi) #setting initial weights
sBA=ones(chi) ./sqrt(chi)

10-element Vector{Float64}:
 0.31622776601683794
 0.31622776601683794
 0.31622776601683794
 0.31622776601683794
 0.31622776601683794
 0.31622776601683794
 0.31622776601683794
 0.31622776601683794
 0.31622776601683794
 0.31622776601683794

In [106]:
sBA = ones(10) ./ sqrt(10)
sigBA=rand(chi,chi)
tol=1e-10
tensors_1 = Any[diagm(0 => sBA),diagm(0 => sBA),A,conj(A), diagm(0 => sAB), diagm(0 => sAB), B, conj(B)]
labels = Any[[1, 2], [1, 3], [2, 4], [3, 5, 6], [4, 5, 7], [6, 8], [7, 9], [8, 10, -1], [9, 10, -2]]
for k in 1:1000
    global sig_BA,tensors_1,labels,tol
    sigBA_new = ncon(Any[sigBA, tensors_1...], labels)
    sigBA_new=sigBA_new/tr(sigBA_new)
    if norm(sigBA-sigBA_new)<tol
        @printf "success! \n"
        break
    else
        @printf "iter: %d, diff: %e \n" k norm(sigBA - sigBA_new)
        sigBA = sigBA_new;
    end
end

iter: 1, diff: 4.585507e+00 
iter: 2, diff: 5.442123e-03 
iter: 3, diff: 9.515539e-05 
iter: 4, diff: 1.628490e-06 
iter: 5, diff: 2.821159e-08 
iter: 6, diff: 5.346380e-10 
success! 


In [107]:
sigBA=rand(chi,chi)
sBA=ones(chi) ./sqrt(chi)
tolerance=1e-10#defining the tolerance reuqired for the operation
tensor=Any[diagm(0=>sBA),diagm(0=>sBA),A,conj(A),diagm(0=>sAB),diagm(0=>sAB),B,conj(B)]
label=Any[[1,2],[1,3],[2,4],[3,5,6],[4,5,7],[6,8],[7,9],[8,10,-1],[9,10,-2]]
#iterating conraction until converged
for k in 1:500
    global sigBA,tolerance,tensor,label
    sigBA_new=ncon(Any[sigBA,tensor...],label)
    siggBA_new=sigBA_new/tr(sigBA_new)
  if norm(sigBA - sigBA_new) < tolerance
    @printf "success! \n"
    break
  else
    @printf "iter: %d, diff: %e \n" k norm(sigBA - sigBA_new)
    sigBA = sigBA_new;
  end
end

iter: 1, diff: 1.376832e+02 
iter: 2, diff: 3.815748e+03 
iter: 3, diff: 1.059058e+05 
iter: 4, diff: 2.939470e+06 
iter: 5, diff: 8.158648e+07 
iter: 6, diff: 2.264474e+09 
iter: 7, diff: 6.285163e+10 
iter: 8, diff: 1.744479e+12 
iter: 9, diff: 4.841890e+13 
iter: 10, diff: 1.343891e+15 
iter: 11, diff: 3.730038e+16 
iter: 12, diff: 1.035291e+18 
iter: 13, diff: 2.873502e+19 
iter: 14, diff: 7.975551e+20 
iter: 15, diff: 2.213655e+22 
iter: 16, diff: 6.144111e+23 
iter: 17, diff: 1.705329e+25 
iter: 18, diff: 4.733228e+26 
iter: 19, diff: 1.313731e+28 
iter: 20, diff: 3.646328e+29 
iter: 21, diff: 1.012057e+31 
iter: 22, diff: 2.809015e+32 
iter: 23, diff: 7.796564e+33 
iter: 24, diff: 2.163976e+35 
iter: 25, diff: 6.006225e+36 
iter: 26, diff: 1.667058e+38 
iter: 27, diff: 4.627004e+39 
iter: 28, diff: 1.284249e+41 
iter: 29, diff: 3.564497e+42 
iter: 30, diff: 9.893442e+43 
iter: 31, diff: 2.745975e+45 
iter: 32, diff: 7.621593e+46 
iter: 33, diff: 2.115412e+48 
iter: 34, diff: 5.8

In [ ]:
# import Pkg
# Pkg.update()

    Updating registry at `C:\Users\shikh\.julia\registries\General.toml`
   Installed GR_jll ──────────────────── v0.73.7+0
   Installed libdecor_jll ────────────── v0.2.2+0
   Installed HypergeometricFunctions ─── v0.3.24
   Installed Accessors ───────────────── v0.1.38
   Installed libfdk_aac_jll ──────────── v2.0.3+0
   Installed StatsFuns ───────────────── v1.3.2
   Installed MutableArithmetics ──────── v1.4.6
   Installed NNlib ───────────────────── v0.9.23
   Installed Opus_jll ────────────────── v1.3.3+0
   Installed Unitful ─────────────────── v1.21.0
   Installed InlineStrings ───────────── v1.4.2
   Installed ConcurrentUtilities ─────── v2.4.2
   Installed StaticArrays ────────────── v1.9.7
   Installed GPUArrays ───────────────── v10.3.1
   Installed RecursiveArrayTools ─────── v3.27.0
   Installed ColorSchemes ────────────── v3.26.0
   Installed Static ──────────────────── v1.1.1
   Installed Bijections ──────────────── v0.1.9
   Installed KrylovKit ───────────────── v0.8.1

In [2]:
using Pkg
Pkg.add("LinearMaps")
Pkg.add("Arpack")
Pkg.add("SparseArrays")

   Resolving package versions...
  No Changes to `C:\Users\shikh\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\shikh\.julia\environments\v1.10\Manifest.toml`
Precompiling project...
  ✓ Libmount_jll
  ✓ EpollShim_jll
  ✓ Xorg_libICE_jll
  ✓ Xorg_libXau_jll
  ✓ gperf_jll
  ✓ mtdev_jll
  ✓ Xorg_libXdmcp_jll
┌ Warning: attempting to remove probably stale pidfile
│   path = "C:\\Users\\shikh\\.julia\\compiled\\v1.10\\DomainSets\\ksPnB_RzhTg.ji.pidfile"
└ @ FileWatching.Pidfile C:\Users\shikh\.julia\juliaup\julia-1.10.0+0.x64.w64.mingw32\share\julia\stdlib\v1.10\FileWatching\src\pidfile.jl:244
  ✓ LinearMaps → LinearMapsChainRulesCoreExt
  ✓ libevdev_jll
  ✓ Wayland_protocols_jll
  ✓ Arpack_jll
  ✓ Xorg_xtrans_jll
┌ Warning: attempting to remove probably stale pidfile
│   path = "C:\\Users\\shikh\\.julia\\compiled\\v1.10\\ITensorsPackageCompilerExt\\Tbb91_RzhTg.ji.pidfile"
└ @ FileWatching.Pidfile C:\Users\shikh\.julia\juliaup\julia-1.10.0+0.x64.w64.mingw32\share\julia\s

In [12]:
using LinearAlgebra
using Arpack
using SparseArrays
chi=16
sX = [0 1; 1 0];
sY = [0 -im; im 0];
htemp = real(kron(sX, sX) + kron(sY, sY));
hamAB = reshape(htemp, 2, 2, 2, 2);
d = size(hamAB, 1);
A_1 = rand(chi,d, d, chi);
# Create a sample matrix (you can replace this with your own matrix)

A=reshape(A_1,d*chi,d*chi)
A = sparse(A)
# Compute eigenvalues and eigenvectors
λ, ϕ = eigs(A, nev=2, which=:LM)

println("Eigenvalues: ", λ)
println("Eigenvectors: ", ϕ)


Eigenvalues: ComplexF64[15.785788577584126 + 0.0im, -0.4490125865794141 + 1.6132873246210775im]
Eigenvectors: ComplexF64[-0.13458202087638063 + 0.0im -0.12503623288707774 + 0.05224630216881391im; -0.1807290407036963 + 0.0im -0.22690895676093809 + 0.1060609556374482im; -0.16909690997771853 + 0.0im 0.06054713588194745 - 0.019912284704086854im; -0.18762440376450698 + 0.0im 0.22662512514483985 + 0.04591708654628731im; -0.21873074978161783 + 0.0im -0.008330367201788803 - 0.09701504709054679im; -0.20351477140424254 + 0.0im -0.14639303348796376 - 0.06269962738829003im; -0.18198167544132696 + 0.0im 0.053897388609255414 + 0.17945356171046484im; -0.1873651602934368 + 0.0im 0.10221685000977161 + 0.03002778816581335im; -0.19237850618189725 + 0.0im -0.2067166029565845 - 0.08608187183997844im; -0.17872430875976975 + 0.0im -0.10444861525874724 - 0.14952431461313867im; -0.16902859449363783 + 0.0im 0.020063349693532264 - 0.029746950355553577im; -0.15909616140129076 + 0.0im -0.14799997532778195 - 0.2721

In [15]:
using LinearMaps,Arpack
chi = 16; # bond dimension
sX = [0 1; 1 0];
sY = [0 -im; im 0];
htemp = real(kron(sX, sX) + kron(sY, sY));
hamAB = reshape(htemp, 2, 2, 2, 2);
hamBA = reshape(htemp, 2, 2, 2, 2);
d = size(hamAB, 1);
sAB = ones(chi) ./ sqrt(chi);
sBA = ones(chi) ./ sqrt(chi);
A = rand(chi, d, chi);
B = rand(chi, d, chi);
sigBA = sigBA = Matrix{Float64}(I,size(A,1),size(A,1)) ./size(A,1);

function left_contract_MPS(sigBA, sBA, A, sAB, B)
    # Initialize the starting vector
    chiBA = size(A, 1);
    if size(sigBA, 1) == size(A, 1)
        v0 = sigBA[:];
    else
        v0 = (Matrix{Float64}(I, chiBA, chiBA) ./ chiBA)[:];
    end

    # Define network for transfer operator contraction
    tensors = Any[diagm(0 => sBA), diagm(0 => sBA), A, conj(A), diagm(0 => sAB),
                  diagm(0 => sAB), B, conj(B)];
    labels = Any[[1, 2], [1, 3], [2, 4], [3, 5, 6], [4, 5, 7], [6, 8], [7, 9], [8, 10, -1],
                  [9, 10, -2]];

    # Define function for boundary contraction and pass to eigs
    function left_iter(sigBA, tensors, labels, chi)
        return reshape(ncon(Any[reshape(sigBA, chi, chi), tensors...], labels), chi^2);
    end

    LI = LinearMap(sigBA -> left_iter(sigBA, tensors, labels, chiBA), chiBA^2;
                   ismutating = false, issymmetric = false, ishermitian = false, isposdef = false)
    Dtemp, sigBA = eigs(LI; v0 = v0, nev = 1, tol = 1e-10, which = :LM, maxiter = 300);

    # Normalize the environment density matrix sigBA
    if eltype(A) == Float64
        sigBA = real(sigBA);
    end
    sigBA = reshape(sigBA, chiBA, chiBA);
    sigBA = 0.5 * (sigBA + sigBA');
    sigBA = sigBA ./ tr(sigBA);

    # Compute density matrix sigBA for B-A link
    sigAB = ncon(Any[sigBA, diagm(0 => sBA), diagm(0 => sBA), A, conj(A)],
                 Any[[1, 2], [1, 3], [2, 4], [3, 5, -1], [4, 5, -2]]);
    sigAB = sigAB ./ tr(sigAB);

    return sigBA, sigAB
end

M = left_contract_MPS(sigBA, sBA, A, sAB, B)

([0.06379059949430722 0.05959274142875258 … 0.06497553158537851 0.05435900726473543; 0.05959274142875258 0.05570574749392575 … 0.06061462462944825 0.050829085422176346; … ; 0.06497553158537851 0.06061462462944825 … 0.0665059220776354 0.05516886647587098; 0.05435900726473543 0.050829085422176346 … 0.05516886647587098 0.04646695410117937], [0.053439533830882686 0.05695906615081699 … 0.056272862115038125 0.054106488383735885; 0.05695906615081698 0.061666204661984435 … 0.06187218224844039 0.057936074506583904; … ; 0.05627286211503812 0.06187218224844036 … 0.06303101062239386 0.05750641269528921; 0.05410648838373588 0.0579360745065839 … 0.05750641269528923 0.054857390792160975])

In [143]:
function otrthogonal_MPS(sigBA,muBA,B,sBA,A,dtol=1e-12)
    #working on left enviroment vector
    dtemp=eigen(0.5*(sigBA+sigBA'));
    ord=sortperm(dtemp.values;rev=true);
    chitemap=sum(dtemp.values .>dtol);
    UL=dtemp.vectors[:,ord[1:chitemap]];
    DL=dtemp.values[ord[1:chitemap]];
    #working on right enviroment vector
    dtemp=eigen(0.5*(muBA+muBA'));
    ord=sortperm(dtemp.values;rev=true);
    chitemap=sum(dtemp.values .>dtol);
    UR=dtemp.vectors[:,ord[1:chitemap]];
    DR=dtemp.values[ord[1:chitemap]];
    #computing for creatign the center of orthogonality
    F=svd(diagm(0=>sqrt.(DL))*conj(UL')*diagm(0=>sBA)*UR*diagm(0=>sqtr.(DR)));
    sBA=F.S/norm(F.S)
    x=conj(UL)*diagm(0=>1 ./sqrt(DL))*F.U;
    y=conj(UR*diagm(0=>1 ./sqrt(DR))*F.V);
    A=ncon(Any[y,A],Any[[1,-1],[1,-2,-3]]);
    B=ncon(Any[B,x],Any[[-1,-2,2],[2,-3]]);
    return B,sBA,A
end

otrthogonal_MPS (generic function with 2 methods)

In [158]:
#applying gate to MPS
function apply_gate(gateAB,sAB,B,sBA,A,chi,stol=1e-7)
    #ensuring for singular values above the tolerance
    sBA_trim=sBA.*(sBA.>stol)+stol.*(sBA.<stol);
    d=size(A,2);
    chiBA=size(sBA_trim,1);
    tensors=Any[diagm(0=>sBA_trim),A,diagm(0=>sAB),B,diagm(0=>sBA_trim),gateAB];
    connects=Any[[-1,1],[1,5,2],[2,4],[4,6,3],[3,-4],[-2,-3,5,6]];
    nshape=[d*chiBA,d*chiBA];
    F=svd(reshape(ncon(tensors,connects),nshape...));
    # truncate to reduced dimension
    chitemp=min(chi,length(F.S));
    utemp=reshape(F.U[:,1:chitemp],size(sBA_trim,1),d*chitemp);
    vhtemp = reshape((F.Vt[1:chitemp,:]),chitemp*d,size(sBA_trim,1))
    A = reshape(diagm(0 => (1 ./sBA_trim))*utemp,chiBA,d,chitemp);
    B = reshape(vhtemp*diagm(0 => (1 ./sBA_trim)),chitemp,d,chiBA);
    sAB = F.S[1:chitemp]./sqrt(sum((F.S[1:chitemp]).^2));
    return A,sAB,B
end
function loc_density_MPS(A,sAB,B,sBA)

  mAB = diagm(0 => sAB);
  mBA = diagm(0 => sBA);

  # contract MPS for local reduced density matrix (A-B)
  tensors = Any[diagm(0 => sBA.^2),A,conj(A),mAB,mAB,B,conj(B),diagm(0 =>sBA.^2)];
  connects = Any[[3,4],[3,-3,1],[4,-1,2],[1,7],[2,8],[7,-4,5],[8,-2,6],[5,6]];
  rhoAB = ncon(tensors,connects);

  # contract MPS for local reduced density matrix (B-A)
  tensors = Any[diagm(0 => sAB.^2),B,conj(B),mBA,mBA,A,conj(A),diagm(0 =>sAB.^2)];
  connects = Any[[3,4],[3,-3,1],[4,-1,2],[1,7],[2,8],[7,-4,5],[8,-2,6],[5,6]];
  rhoBA = ncon(tensors,connects);

  return rhoAB, rhoBA
end

loc_density_MPS (generic function with 1 method)

In [ ]:
function loc_density_MPS(A,sAB,B,sBA)

  mAB = diagm(0 => sAB);
  mBA = diagm(0 => sBA);

  # contract MPS for local reduced density matrix (A-B)
  tensors = Any[diagm(0 => sBA.^2),A,conj(A),mAB,mAB,B,conj(B),diagm(0 =>sBA.^2)];
  connects = Any[[3,4],[3,-3,1],[4,-1,2],[1,7],[2,8],[7,-4,5],[8,-2,6],[5,6]];
  rhoAB = ncon(tensors,connects);

  # contract MPS for local reduced density matrix (B-A)
  tensors = Any[diagm(0 => sAB.^2),B,conj(B),mBA,mBA,A,conj(A),diagm(0 =>sAB.^2)];
  connects = Any[[3,4],[3,-3,1],[4,-1,2],[1,7],[2,8],[7,-4,5],[8,-2,6],[5,6]];
  rhoBA = ncon(tensors,connects);

  return rhoAB, rhoBA

In [178]:
using Pkg
Pkg.add("Arpack")

   Resolving package versions...
  No Changes to `C:\Users\shikh\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\shikh\.julia\environments\v1.10\Manifest.toml`


In [16]:
using Printf, LinearAlgebra, LinearMaps, Arpack
function doTEBD(hamAB,hamBA,A,B,sAB,sBA,chi,tau; evotype = "imag",
  numiter = 1000, midsteps = 10, E0 = 0.0)

  # exponentiate Hamiltonian
  d = size(A,2);
  if evotype == "real"
    gateAB = reshape(exp(im*tau*reshape(hamAB,d^2,d^2)),d,d,d,d);
    gateBA = reshape(exp(im*tau*reshape(hamBA,d^2,d^2)),d,d,d,d);
  elseif evotype == "imag"
    gateAB = reshape(exp(-tau*reshape(hamAB,d^2,d^2)),d,d,d,d);
    gateBA = reshape(exp(-tau*reshape(hamBA,d^2,d^2)),d,d,d,d);
  end

  # initializr environment tensors
  sigBA = Matrix{Float64}(I,size(A,1),size(A,1)) ./size(A,1);
  muAB = Matrix{Float64}(I,size(A,3),size(A,3)) ./size(A,3);

  for k = 0:numiter
    if mod(k,midsteps) == 0 || (k == numiter)
      """ Bring MPS to normal form """

      # contract MPS from the left and right
      sigBA,sigAB = left_contract_MPS(sigBA,sBA,A,sAB,B)
      muAB,muBA = right_contract_MPS(muAB,sBA,A,sAB,B)

      # orthogonalise A-B and B-A links
      B, sBA, A = orthog_MPS(sigBA,muBA,B,sBA,A);
      A, sAB, B = orthog_MPS(sigAB,muAB,A,sAB,B);

      # normalize the MPS tensors
      A_norm = sqrt(ncon(Any[diagm(0 => sBA.^2),A,conj(A),diagm(0 => sAB.^2)],
        [[1,3],[1,4,2],[3,4,5],[2,5]]));
      A = A./A_norm;
      B_norm = sqrt(ncon(Any[diagm(0 => sAB.^2),B,conj(B),diagm(0 => sBA.^2)],
        [[1,3],[1,4,2],[3,4,5],[2,5]]));
      B = B./B_norm;

      """ Compute energy and display """

      # compute 2-site local reduced density matrices
      rhoAB, rhoBA = loc_density_MPS(A,sAB,B,sBA);

      # evaluate the energy
      energyAB = ncon(Any[hamAB,rhoAB],Any[[1,2,3,4],[1,2,3,4]]);
      energyBA = ncon(Any[hamBA,rhoBA],Any[[1,2,3,4],[1,2,3,4]]);
      energy = 0.5*(energyAB + energyBA);

      # print to display
      chi_temp = min(size(A,1),size(B,1));
      @printf "Iteration: %d of %d, Bond dim: %d, " k numiter  chi_temp
      @printf "t-step: %e, Energy: %f, Energy Error: %e\n" tau energy energy-E0
  end

    """ Do evolution of MPS through one time-step """
    if k < numiter
      # apply gate to A-B link
      A, sAB, B = apply_gate_MPS(gateAB,A,sAB,B,sBA,chi)
      # apply gate to B-A link
      B, sBA, A = apply_gate_MPS(gateBA,B,sBA,A,sAB,chi)
    end
  end

  rhoAB, rhoBA = loc_density_MPS(A,sAB,B,sBA);
  return A, B, sAB, sBA, rhoAB, rhoBA
end


"""
left_contract_MPS: Contract an infinite 2-site unit cell from the left for
the environment density matrices sigBA (B-A link) and sigAB (A-B link)
"""
function left_contract_MPS(sigBA,sBA,A,sAB,B)

  # initialize the starting vector
  chiBA = size(A,1);
  if size(sigBA,1) == size(A,1)
    v0 = sigBA[:];
  else
    v0 = (Matrix{Float64}(I,chiBA,chiBA) ./chiBA)[:];
  end

  # define network for transfer operator contraction
  tensors = Any[diagm(0 => sBA),diagm(0 => sBA),A,conj(A),diagm(0 => sAB),
    diagm(0 => sAB),B,conj(B)];
  labels = Any[[1,2],[1,3],[2,4],[3,5,6],[4,5,7],[6,8],[7,9],[8,10,-1],
    [9,10,-2]];

  # define function for boundary contraction and pass to eigs
  function left_iter(sigBA,tensors,labels,chi)
    return reshape(ncon(Any[reshape(sigBA,chi,chi),tensors...],labels),chi^2);
  end
  LI = LinearMap(sigBA -> left_iter(sigBA,tensors,labels,chiBA), chiBA^2;
    ismutating=false, issymmetric=false, ishermitian=false, isposdef=false)
  Dtemp, sigBA = eigs(LI; v0 = v0,nev = 1,tol = 1e-10,which=:LM,maxiter = 300);

  # normalize the environment density matrix sigBA
  if eltype(A) == Float64
    sigBA = real(sigBA);
  end
  sigBA = reshape(sigBA,chiBA,chiBA);
  sigBA = 0.5*(sigBA + sigBA');
  sigBA = sigBA ./tr(sigBA);

  # compute density matrix sigBA for B-A link
  sigAB = ncon(Any[sigBA,diagm(0 => sBA),diagm(0 => sBA),A,conj(A)],
    Any[[1,2],[1,3],[2,4],[3,5,-1],[4,5,-2]]);
  sigAB = sigAB./tr(sigAB);

  return sigBA,sigAB
end


"""
right_contract_MPS: Contract an infinite 2-site unit cell from the right for
the environment density matrices muBA (B-A link) and muAB (A-B link)
"""
function right_contract_MPS(muAB,sBA,A,sAB,B)

  # initialize the starting vector
  chiAB = size(A,3);
  if size(muAB,1) == chiAB
    v0 = muAB[:];
  else
    v0 = (Matrix{Float64}(I,chiAB,chiAB) ./chiAB)[:];
  end

  # define network for transfer operator contraction
  tensors = Any[diagm(0 => sAB),diagm(0 => sAB),A,conj(A),diagm(0 => sBA),
    diagm(0 => sBA),B,conj(B)];
  labels = Any[[1,2],[3,1],[5,2],[6,4,3],[7,4,5],[8,6],[10,7],[-1,9,8],
    [-2,9,10]];

  # define function for boundary contraction and pass to eigs
  function right_iter(muAB,tensors,labels,chiAB)
    return reshape(ncon(Any[reshape(muAB,chiAB,chiAB),tensors...],labels),
      chiAB^2);
  end
  RI = LinearMap(muAB -> right_iter(muAB,tensors,labels,chiAB), chiAB^2;
    ismutating=false, issymmetric=false, ishermitian=false, isposdef=false)
  Dtemp, muAB = eigs(RI; v0 = v0,nev = 1,tol = 1e-10,which=:LM,maxiter = 300);

  # normalize the environment density matrix muAB
  if eltype(A) == Float64
    muAB = real(muAB);
  end
  muAB = reshape(muAB,chiAB,chiAB);
  muAB = 0.5*(muAB + muAB');
  muAB = muAB ./tr(muAB);

  # compute density matrix muBA for B-A link
  muBA = ncon(Any[muAB,diagm(0 => sAB),diagm(0 => sAB),A,conj(A)],
    Any[[1,2],[3,1],[5,2],[-1,4,3],[-2,4,5]]);
  muBA = muBA ./tr(muBA);

  return muAB,muBA
end

"""
orthog_MPS: Set the MPS gauge across B-A link to the canonical form
"""
function orthog_MPS(sigBA,muBA,B,sBA,A; dtol = 1e-12)

  # diagonalize left environment tensor
  dtemp = eigen(0.5*(sigBA+sigBA'));
  ord = sortperm(dtemp.values; rev=true);
  chitemp = sum(dtemp.values .> dtol);
  UL = dtemp.vectors[:,ord[1:chitemp]];
  DL = dtemp.values[ord[1:chitemp]];

  # diagonalize right environment tensor
  dtemp = eigen(0.5*(muBA+muBA'));
  ord = sortperm(dtemp.values; rev=true);
  chitemp = sum(dtemp.values .> dtol);
  UR = dtemp.vectors[:,ord[1:chitemp]];
  DR = dtemp.values[ord[1:chitemp]];

  # compute new weights for B-A link
  F = svd(diagm(0 => sqrt.(DL))*conj(UL')*
    diagm(0 => sBA)*UR*diagm(0 => sqrt.(DR)));
  sBA = F.S ./ norm(F.S)

  # build x,y gauge change matrices, implement gauge change on A and B
  x = conj(UL)*diagm(0 => 1 ./sqrt.(DL))*F.U;
  y = conj(UR*diagm(0 => 1 ./sqrt.(DR))*F.V);
  A = ncon(Any[y,A],Any[[1,-1],[1,-2,-3]]);
  B = ncon(Any[B,x],Any[[-1,-2,2],[2,-3]]);

  return B, sBA, A
end

"""
apply_gate_MPS: Apply a gate to an MPS across and a A-B link. Truncate the MPS
back to some desired dimension chi.
"""
function apply_gate_MPS(gateAB,A,sAB,B,sBA,chi,stol=1e-7)

  # ensure singular values are above tolerance threshold
  sBA_trim = sBA.*(sBA .> stol) + stol.*(sBA .< stol);

  # contract gate into the MPS, then deompose composite tensor with SVD
  d = size(A,2);
  chiBA = size(sBA_trim,1);
  tensors = Any[diagm(0=>sBA_trim),A,diagm(0=>sAB),B,diagm(0=>sBA_trim),gateAB];
  connects = Any[[-1,1],[1,5,2],[2,4],[4,6,3],[3,-4],[-2,-3,5,6]];
  nshape = [d*chiBA,d*chiBA];
  F = svd(reshape(ncon(tensors,connects),nshape...));

  # truncate to reduced dimension
  chitemp = min(chi,length(F.S));
  utemp = reshape(F.U[:,1:chitemp],size(sBA_trim,1),d*chitemp);
  vhtemp = reshape((F.Vt[1:chitemp,:]),chitemp*d,size(sBA_trim,1))

  # remove environment weights to form new MPS tensors A and B
  A = reshape(diagm(0 => (1 ./sBA_trim))*utemp,chiBA,d,chitemp);
  B = reshape(vhtemp*diagm(0 => (1 ./sBA_trim)),chitemp,d,chiBA);

  # new weights
  sAB = F.S[1:chitemp]./sqrt(sum((F.S[1:chitemp]).^2));

  return A,sAB,B
end

"""
loc_density_MPS: compute the local reduced density matrices from an MPS
(assumend to be in canonical form).
"""
function loc_density_MPS(A,sAB,B,sBA)

  mAB = diagm(0 => sAB);
  mBA = diagm(0 => sBA);

  # contract MPS for local reduced density matrix (A-B)
  tensors = Any[diagm(0 => sBA.^2),A,conj(A),mAB,mAB,B,conj(B),diagm(0 =>sBA.^2)];
  connects = Any[[3,4],[3,-3,1],[4,-1,2],[1,7],[2,8],[7,-4,5],[8,-2,6],[5,6]];
  rhoAB = ncon(tensors,connects);

  # contract MPS for local reduced density matrix (B-A)
  tensors = Any[diagm(0 => sAB.^2),B,conj(B),mBA,mBA,A,conj(A),diagm(0 =>sAB.^2)];
  connects = Any[[3,4],[3,-3,1],[4,-1,2],[1,7],[2,8],[7,-4,5],[8,-2,6],[5,6]];
  rhoBA = ncon(tensors,connects);

  return rhoAB, rhoBA
end


loc_density_MPS

In [17]:
  
"""
mainTEBD.jl
---------------------------------------------------------------------
Script file for initializing the Hamiltonian and MPS tensors before passing to
the TEBD routine.

by Glen Evenbly (c) for www.tensors.net, (v1.2) - last modified 06/2020
"""

using Printf, LinearAlgebra, LinearMaps, Arpack



##### Example 1: XX model #############
#######################################

##### Set bond dimensions and simulation options
chi = 16; # bond dimension
tau = 0.1; # timestep

numiter = 500; # number of timesteps
evotype = "imag"; # real or imaginary time evolution
E0 = -4/pi; # specify exact ground energy (if known)
midsteps = Int64(1/tau); # timesteps between MPS re-orthogonalization

#### Define Hamiltonian (quantum XX model)
sX = [0 1; 1 0]; sY = [0 -im; im 0]; sZ = [1 0; 0 -1]; sI = [1 0; 0 1];
htemp = real(kron(sX,sX) + kron(sY,sY));
hamAB = reshape(htemp,2,2,2,2);
hamBA = reshape(htemp,2,2,2,2);

##### Initialize tensors
d = size(hamAB,1);
sAB = ones(chi) ./sqrt(chi);
sBA = ones(chi) ./sqrt(chi);
A = rand(chi,d,chi);
B = rand(chi,d,chi);

##### Imaginary time evolution with TEBD ######
###############################################

##### Run TEBD routine
A, B, sAB, sBA, rhoAB, rhoBA = doTEBD(hamAB,hamBA,A,B,sAB,sBA,chi,tau;
  evotype = evotype, numiter = numiter, midsteps = midsteps, E0 = E0);

##### continute running TEBD routine with reduced timestep
tau = 0.01;
numiter = 2000;
midsteps = 100;
A, B, sAB, sBA, rhoAB, rhoBA = doTEBD(hamAB,hamBA,A,B,sAB,sBA,chi,tau;
  evotype = evotype, numiter = numiter, midsteps = midsteps, E0 = E0);

##### continute running TEBD with reduced timestep and increased bond dim
chi = 32;
tau = 0.001;
numiter = 20000;
midsteps = 1000;
A, B, sAB, sBA, rhoAB, rhoBA = doTEBD(hamAB,hamBA,A,B,sAB,sBA,chi,tau;
  evotype = evotype, numiter = numiter, midsteps = midsteps, E0 = E0);

##### Compare with exact results
energyAB = ncon(Any[hamAB,rhoAB],Any[[1,2,3,4],[1,2,3,4]]);
energyBA = ncon(Any[hamBA,rhoBA],Any[[1,2,3,4],[1,2,3,4]]);
energyMPS = 0.5*(energyAB + energyBA)
enErr = abs(energyMPS + 4/pi)
@printf "Bond dim: %d, Energy: %f, Energy Error: %e\n" chi energyMPS enErr


Iteration: 0 of 500, Bond dim: 16, t-step: 1.000000e-01, Energy: 1.002415, Energy Error: 2.275654e+00
Iteration: 10 of 500, Bond dim: 16, t-step: 1.000000e-01, Energy: -1.251057, Energy Error: 2.218277e-02
Iteration: 20 of 500, Bond dim: 16, t-step: 1.000000e-01, Energy: -1.262672, Energy Error: 1.056736e-02
Iteration: 30 of 500, Bond dim: 16, t-step: 1.000000e-01, Energy: -1.263943, Energy Error: 9.297023e-03
Iteration: 40 of 500, Bond dim: 16, t-step: 1.000000e-01, Energy: -1.264330, Energy Error: 8.909880e-03
Iteration: 50 of 500, Bond dim: 16, t-step: 1.000000e-01, Energy: -1.264484, Energy Error: 8.755836e-03
Iteration: 60 of 500, Bond dim: 16, t-step: 1.000000e-01, Energy: -1.264554, Energy Error: 8.685490e-03
Iteration: 70 of 500, Bond dim: 16, t-step: 1.000000e-01, Energy: -1.264589, Energy Error: 8.650580e-03
Iteration: 80 of 500, Bond dim: 16, t-step: 1.000000e-01, Energy: -1.264607, Energy Error: 8.632297e-03
Iteration: 90 of 500, Bond dim: 16, t-step: 1.000000e-01, Energy: 

In [110]:
arr = [1, 2, 2, 3, 4, 4, 5,65,65,65,89,89,9878,56,98,898,6,4,464,64,66,46,]
unique_arr = unique(arr)
println(unique_arr)  # Output: [1, 2, 3, 4, 5]

[1, 2, 3, 4, 5, 65, 89, 9878, 56, 98, 898, 6, 464, 64, 66, 46]


In [111]:
s = [[4, 5], [6, 5, 8], [7, 8, 9], [4, 6]]
for i in 1:length(s)
    println(length(s[i]))
end


2
3
3
2


* **TUTORIAL-1**

**Time evolution practice**

In [175]:
sx=[0 1;1 0];
sy=[0 -im;im 0];
htemp=real(kron(sx,sx)+kron(sy,sy));
hamAB=reshape(htemp,2,2,2,2);
hamBA=reshape(htemp,2,2,2,2);
#exponentiate Hamiltonian
d=size(A,2)
tau=0.1
evotype="imag";
if evotype=="real"
    gateAB=reshape(exp(im*tau*reshape(hamAB,d^2,d^2)),d,d,d,d);
    gateBA=reshape(exp(im*tau*reshape(hamBA,d^2,d^2)),d,d,d,d);
elseif evotype=="imag"
    gateAB=reshape(exp(-tau*reshape(hamAB,d^2,d^2)),d,d,d,d);
    gateBA=reshape(exp(-tau*reshape(hamBA,d^2,d^2)),d,d,d,d); 
end

2×2×2×2 Array{Float64, 4}:
[:, :, 1, 1] =
 1.0  0.0
 0.0  0.0

[:, :, 2, 1] =
 0.0      -0.201336
 1.02007   0.0

[:, :, 1, 2] =
  0.0       1.02007
 -0.201336  0.0

[:, :, 2, 2] =
 0.0  0.0
 0.0  1.0

In [112]:
using LinearAlgebra
A=rand(2,4,3)
B=Matrix{Float64}(I,5,5)#defining a tensor of identity matrix of 5x5
C=ones(2,4,2,4)#defining a tensor of 4 rank
D=zeros(5,5)
#defining a comples random tensor
E=rand(2,3,4)+im*rand(2,3,4)

2×3×4 Array{ComplexF64, 3}:
[:, :, 1] =
 0.453069+0.220679im  0.116454+0.277262im   0.701532+0.323836im
 0.832515+0.424794im  0.525845+0.242506im  0.0599013+0.248022im

[:, :, 2] =
 0.086192+0.328428im  0.449395+0.620913im   0.452569+0.685306im
 0.508348+0.790195im  0.484215+0.352661im  0.0116235+0.0999356im

[:, :, 3] =
 0.707923+0.589519im  0.568912+0.74252im   0.616823+0.688158im
 0.802945+0.353526im  0.753319+0.762892im  0.711618+0.284652im

[:, :, 4] =
 0.770043+0.5572im     0.511858+0.148073im  0.639779+0.819103im
 0.612072+0.0758381im  0.701788+0.778884im  0.782202+0.119688im

In [113]:
d=10
A=rand(d,d,d,d)
B=rand(d,d,d,d)
Ap=permutedims(A,[1,3,2,4]);Bp=permutedims(B,[1,4,2,3]);
App=reshape(A,(d^2,d^2));Bpp=reshape(B,(d^2,d^2));
Cpp=App*Bpp
C=reshape(Cpp,(d,d,d,d));

In [114]:
d=10
A=rand(d,d);B=rand(d,d);C=rand(d,d);
F0=zeros(d,d);
for id in 1:d
    for jd in 1:d
        for kd in 1:d
            for ld in 1:d
                F0[id,jd]=F0[id,jd]+A[id,kd]*B[kd,ld]*C[ld,jd]
            end
        end
    end
end
F0

10×10 Matrix{Float64}:
 11.4688  10.1632   8.57089   9.28719  …   8.63705   6.6456    9.83561
 17.1444  14.6498  12.7134   14.1708      13.3896    9.75644  14.218
 19.3534  16.8736  15.0027   15.6411      15.6983   11.5255   15.8316
 21.66    18.8352  16.58     17.8208      17.3262   12.7464   17.6924
 24.1789  20.8568  18.4563   19.4006      19.1936   14.1565   19.9819
 16.7738  14.458   12.5161   13.4667   …  12.9672    9.65061  14.1304
 23.3528  20.7061  17.9347   18.9533      18.6326   13.9591   18.8435
 17.1581  14.9423  12.9129   13.774       13.1865   10.0222   14.468
 21.7324  18.7026  16.335    17.2192      16.8909   12.7022   18.0268
 16.1981  14.0095  12.1506   12.7867      12.6927    9.36727  13.3798

In [115]:
d=10
a=rand(d,d,d);b=rand(d,d,d,d);c=rand(d,d,d);d=rand(d,d);
TensorArray=Any[a,b,c,d];
IndexArray=Any[[1,-2,2],[-1,1,3,4],[5,3,2],[4,5]]
cont_order=[5,3,4,2,1]
E=ncon(TensorArray,IndexArray)
size(E)

(10, 10)

In [116]:
d=10
A=rand(d,d,d,d,d,d);
B=ncon(Any[A],Any[-1,-2,1,-3,-4,1]);

In [117]:
A=rand(d,d)
B=rand(d,d)
C=ncon(Any[A,B],Any[[-1,-2],[-3,-4]]);

In [118]:
d=10
A=rand(d,d,d);B=rand(d,d,d,d);C=rand(d,d,d,d);D=rand(d,d,d);E=rand(d,d,d);
TensorArray=Any[A,B,C,D,E];
IndexArray=Any[[-1,1,2],[1,-2,3,7],[6,2,7,4],[-3,6,5],[5,4,3]];
F=ncon(TensorArray,IndexArray);
size(F)

(10, 10, 10)

In [119]:

d = 20;
A = rand(d,d,d); B = rand(d,d,d); C = rand(d,d,d);

##### Evaluate network via index summation
function tempfunct(A,B,C,d)
    D0 = zeros(d,d,d);
    for b1 = 1:d
        for a2 = 1:d
            for c3 = 1:d
                for a1 = 1:d
                    for a3 = 1:d
                        for c1 = 1:d
                            D0[b1,a2,c3] = D0[b1,a2,c3]+A[a1,a2,a3]*B[b1,a1,c1]*C[c1,a3,c3];
                        end
                    end
                end
            end
        end
    end
    return D0
end

t_sum = @elapsed D0 = tempfunct(A,B,C,d);

In [120]:
X=reshape(reshape(permutedims(B,[1,3,2]),(d^2,d))*reshape(A,(d,d^2)),(d,d,d,d));
D0=reshape(reshape(permutedims(X,[1,3,2,4]),(d^2,d^2))*reshape(C,(d^2,d)),(d,d,d));
t2=@elapsed D0
C0=ncon(Any[A,B,C],Any[[1,-2,2],[-1,1,3],[3,2,-3]]);
t3=@elapsed C0
tdiffs=[maximum(abs.(F0[:]-D0[:])),maximum(abs.(D0[:]-C0[:])),maximum(abs.(C0[:]-F0[:]))]

LoadError: DimensionMismatch: dimensions must match: a has dims (Base.OneTo(100),), b has dims (Base.OneTo(8000),), mismatch at 1

In [121]:
t_sum,t2,t3

(0.1157549, 2.0e-7, 2.0e-7)

In [122]:
#calculation of a SVD of a matrix

In [123]:
d1 = 10; d2 = 6;
A=rand(d1,d2);
F=svd(A);
#checking for the correctness
Af=F.U*diagm(0=>F.S)F.Vt;
Da=norm(Af[:]-A[:])

3.520351929799884e-15

In [124]:
d = 10; A = rand(d,d,d);
F = svd(reshape(A,d^2,d));
U = reshape(F.U,d,d,d);
# check result
Af = ncon(Any[U,diagm(0 => F.S),F.Vt], Any[[-1,-2,1],[1,2],[2,-3]]);
diff=norm(Af[:]-A[:])

2.3703674486226177e-14

In [125]:
d=10;A=rand(d,d);
H=0.5*(A+A')
F=eigen(H);
Af=F.vectors*diagm(0=>F.values)*F.vectors';
diff=norm(Af[:]-H[:])

4.979921898473019e-15

In [126]:
#application on tensors
d=2;A=rand(d,d,d,d);
H=0.5(A+permutedims(A,[3,4,1,2]));
F=eigen(reshape(H,d^2,d^2));
U=reshape(F.vectors,d,d,d^2);
S=diagm(0=>F.values);
Af=ncon(Any[U,S,U],Any[[-1,-2,1],[1,2],[-3,-4,2]]);#beacuse of being transpose the last index gets reversed
diff=norm(Af[:]-H[:])

3.2807982676768444e-15

In [127]:
d=10
A=rand(d,d,d,d);
Frob0A=sqrt(ncon(Any[A,conj(A)],Any[collect(1:ndims(A)),collect(1:ndims(A))]))
#equivalent frobenius norm
frob1A=sqrt(sum(abs.(A[:]).^2))
#equivalent frobenius form
frob2A=norm(A)

57.81828997814314

In [128]:
#forming another tensor that close approximates the previous one
d = 10; A = rand(d,d,d,d,d);
F = svd(reshape(A,d^3,d^2));
U = reshape(F.U,d,d,d,d^2);
Vh = reshape(F.Vt,d^2,d,d);
##### truncate SVD
chi = 8;
Vhtilda = Vh[1:chi,:,:];
Stilda = diagm(0 => F.S[1:chi]);
Utilda = U[:,:,:,1:chi];
B = ncon(Any[Utilda,Stilda,Vhtilda],Any[[-1,-2,-3,1],[1,2],[2,-4,-5]]);
epsAB=norm(A-B)/norm(A)

0.46832273608120667

In [129]:
d=500
A=diagm(1=>ones(d))+diagm(-1=>ones(d));
A=A/norm(A);
deleteval=1e-2
F=svd(A);
r_delete=sum(F.S .>deleteval)
eps_erro=sqrt.(sum(F.S[r_delete:end].^2))

0.04274636292626995

**Gauge Freedom**

In [130]:
d = 3;
A = rand(d,d,d,d); B = rand(d,d,d);
C = rand(d,d,d); D = rand(d,d,d);
E = rand(d,d,d); F = rand(d,d,d);
G = rand(d,d,d);
DQ, DR = qr(reshape(D,d^2,d)); DQ = reshape(Matrix(DQ),d,d,d);
EQ, ER = qr(reshape(E,d^2,d)); EQ = reshape(Matrix(EQ),d,d,d);
Btilda = ncon(Any[B,DR,ER],Any[[1,2,-3],[-1,1],[-2,2]]);
BQ, BR = qr(reshape(Btilda,d^2,d)); BQ = reshape(Matrix(BQ),d,d,d);
FQ, FR = qr(reshape(F,d^2,d)); FQ = reshape(Matrix(FQ),d,d,d);
GQ, GR = qr(reshape(G,d^2,d)); GQ = reshape(Matrix(GQ),d,d,d);
Ctilda = ncon(Any[C,GR],Any[[1,-2,-3],[-1,1]]);
CQ, CR = qr(reshape(Ctilda,d^2,d)); CQ = reshape(Matrix(CQ),d,d,d);
Aprime = ncon(Any[A,BR,FR,CR],Any[[1,-2,2,3],[-1,1],[-3,2],[-4,3]]);
H_old=ncon(Any[A,B,C,D,E,F,G],Any[[3,-5,4,5],[1,2,3],[6,-10,5],[-1,-2,1],[-3,-4,2],[-6,-7,4],[-8,-9,6]]);
connect_list=Any[[3,-5,4,5],[1,2,3],[6,-10,5],[-1,-2,1],[-3,-4,2],[-6,-7,4],[-8,-9,6]]
H_new=ncon(Any[Aprime,BQ,CQ,DQ,EQ,FQ,GQ],connect_list[:]);
dh=norm(H_old-H_new)/norm(H_old)

5.389663517066005e-16

**Creating center of orthogonality using direct orthogonalization**

In [131]:
d = 3;
A = rand(d,d,d,d); B = rand(d,d,d);
C = rand(d,d,d); D = rand(d,d,d);
E = rand(d,d,d); F = rand(d,d,d);
G = rand(d,d,d);
rho1=ncon(Any[B,D,E,B,D,E],Any[[5,6,-2],[1,2,5],[3,4,6],[7,8,-1],[1,2,7],[3,4,8]]);
rho2=ncon(Any[F,F],Any[[1,2,-2],[1,2,-1]]);
rho3 = ncon(Any[C,G,C,G],Any[[3,5,-2],[1,2,3],[4,5,-1],[1,2,4]]);
d1,u1=eigen(rho1);sqd1=sqrt.(abs.(d1));
d2,u2=eigen(rho2);sqd2=sqrt.(abs.(d2));
d3,u3=eigen(rho3);sqd3=sqrt.(abs.(d3));
X1=u1*diagm(0=>sqd1)*u1';X1inv=u1*diagm(0=>(1 ./sqd1))*u1';
X2=u2*diagm(0=>sqd2)*u2';X2inv=u2*diagm(0=>(1 ./sqd2))*u2';
X3=u3*diagm(0=>sqd3)*u3';X3inv=u3*diagm(0=>(1 ./sqd3))*u3';
Aprime=ncon(Any[A,X1,X2,X3],Any[[1,-2,2,3],[-1,1],[-3,2],[-4,3]]);
Bprime = ncon(Any[B,X1inv],Any[[-1,-2,1],[1,-3]]);
Fprime = ncon(Any[F,X2inv],Any[[-1,-2,1],[1,-3]]);
Cprime = ncon(Any[C,X3inv],Any[[-1,-2,1],[1,-3]]);
H_new=ncon(Any[Aprime,Bprime,Cprime,D,E,Fprime,G],connect_list[:]);
H_old=ncon(Any[A,B,C,D,E,F,G],connect_list[:]);
dH=norm(H_new-H_old)/norm(H_old)

1.4636427572484577e-15

In [132]:
d=5
H0=reshape(sqrt.(1:d^7),d,d,d,d,d,d,d);
chi=3#internal dimension
#first decomposition
utemp,stemp,vtemp=svd(reshape(H0,d^2,d^5));
U0=reshape(utemp[:,1:chi],d,d,chi);
H1=reshape(diagm(0=>stemp[1:chi])*vtemp[:,1:chi]',chi,d,d,d,d,d);
utemp,stemp,vtemp = svd(reshape(permutedims(H1,[2,3,1,4,5,6]),d^2,chi*d^3));
U1 = reshape(utemp[:,1:chi],d,d,chi);
H2=permutedims(reshape(diagm(0=>stemp[1:chi])*vtemp[:,1:chi]',chi,chi,d,d,d),[2,1,3,4,5]);
utemp,stemp,vtemp=svd(reshape(H2,chi^2,d^3));
U2=reshape(utemp[:,1:chi],chi,chi,chi);
H3=permutedims(reshape(diagm(0=>stemp[1:chi])*vtemp[:,1:chi]',chi,d,d,d),[2,3,1,4]);
utemp,stemp,vtemp=svd(reshape(H3,chi*d,d^2));
V3=reshape(vtemp[:,1:chi],d,d,chi);
H4=reshape(utemp[:,1:chi]*diagm(0=>stemp[1:chi]),chi,d,chi);
H0_recovery=ncon(Any[U0,U1,U2,V3,H4],Any[[-1,-2,1],[-3,-4,2],[1,2,3],[-6,-7,4],[3,-5,4]]);
tot_var=norm(H0-H0_recovery)/norm(H0)

1.153451471243839

In [133]:
#setting the BC link as the center of orthogonality
d=5
A=rand(d,d,d);B=rand(d,d,d);C=rand(d,d,d);
Sig=Matrix{Float64}(I,d,d);#initial link matrix
#generate gauge change matrices
rho1=ncon(Any[A,A,B,B],Any[[1,2,3],[1,2,4],[3,5,-1],[4,5,-2]]);
rho2=ncon(Any[C,C],Any[[1,2,-1],[-2,1,2]]);
d1,u1=eigen(rho1);sq_d1=sqrt.(abs.(d1));
d2,u2=eigen(rho2);sq_d2=sqrt.(abs.(d2));
X1=u1*diagm(0=>sq_d1)*u1';X1inv=u1*diagm(0=> 1 ./sq_d1)*u1';
X2=u2*diagm(0=>sq_d2)*u2';X2inv=u2*diagm(0=>1  ./sq_d2)*u2';
Bprime=ncon(Any[B,X1inv],Any[[-1,-2,1],[1,-3]]);
Cprime=ncon(Any[C,X2inv],Any[[-1,-2,1],[1,-3]]);
sig_prime=X1*Sig*X2;
H0 = ncon(Any[A,B,C],Any[[-1,-2,1],[1,-3,2],[2,-4,-5]]);
H1=ncon(Any[A,Bprime,sig_prime,Cprime],Any[[-1,-2,1],[1,-3,2],[2,3],[3,-4,-5]]);
norm(H1-H0)/norm(H0)

3.739684666310792

In [134]:
Utemp,sig_PP,Vtemp=svd(sig_prime);
B_pp=ncon(Any[Bprime,Utemp],Any[[-1,-2,1],[1,-3]]);
C_pp=ncon(Any[Cprime,Vtemp],Any[[1,-2,-3],[1,-1]]);
H2=ncon(Any[A,B_pp,diagm(0=>sig_PP),C_pp],Any[[-1,-2,1],[1,-3,2],[2,3],[3,-4,-5]]);
norm(H2-H0)/norm(H0)

3.739684666310789

In [135]:
##### Ex4.2(c): equivalence of center of orthogonality and SVD
##### (continues from Ex4.2(b))
H = ncon(Any[A,B,C],Any[[-1,-2,1],[1,-3,2],[2,-4,-5]]);
utemp,stemp,vtemp = svd(reshape(H,d^3,d^2));
UH = reshape(utemp[:,1:d],d,d,d,d);
SH = diagm(0 => stemp[1:d]);
VH = reshape(vtemp[:,1:d]',d,d,d);

# compare with previous tensors from orthonormal form
ErrU = norm(abs.(UH[:]) - abs.(reshape(ncon(Any[A,B_pp],Any[[-1,-2,1],[1,-3,-4]]),d^4)))
ErrS = norm(diag(SH) - sig_PP)
ErrV = norm(abs.(VH[:]) - abs.(C_pp[:]))
# all three results should be vanishingly small!!!
print(ErrU,ErrS,ErrV)

1.53071215025967036.5598230667142686.201391655445346

In [136]:
size(permutedims(H1,[2,3,1,4,5,6]))
size(utemp)
size(diagm(0=>stemp[1:chi])*vtemp[:,1:chi]')

LoadError: ArgumentError: no valid permutation of dimensions

In [137]:
size(diagm(0=>stemp[1:chi])*vtemp[:,1:chi]')
size(utemp)

(125, 25)

In [138]:
size(diagm(0=>stemp[1:chi])*vtemp[:,1:chi]')

(3, 25)